# Packing & Design

Rosetta uses a Monte Carlo optimization routine to pack side chains using a library of conformations, or rotamers. This operation can be used for side-chain packing for operations like refinement or for designing optimal sequences. This workshop will examine both capabilities.

We will also cover many more ways to do protein design within Rosetta including parametric, denovo, and hydrogen-bond based design.  All of these can be useful tools for protein engineering.  

### Suggested readings
1. J. Desmet et al., “The dead-end elimination theorem and its use in protein side-chain positioning,” *Nature* 356, 539-543 (1992).
2. B. Kuhlman & D. Baker, “Native protein structures are close to optimal for their structures,” *PNAS* 97, 10383, 2000.
3. Denovo paper
4. HbNet
5. Andrew's packing paper
6. Denovo design
7. Parametric design paper

**Chapter contributors:**

- Jared Adolf-Bryfogle (Scripps; Institute for Protein Innovation)
- Jason C. Klima (University of Washington; Lyell Immunopharma)
- Jack Maguire (University of North Carolina)
- Kathy Le (Johns Hopkins University); parts of this chapter were adapted from the [PyRosetta book](https://www.amazon.com/PyRosetta-Interactive-Platform-Structure-Prediction-ebook/dp/B01N21DRY8) (J. J. Gray, S. Chaudhury, S. Lyskov, J. Labonte).

# Side Chain Conformations and Dunbrack Energies
Keywords: phi(), psi(), energies(), residue_total_energies()

# References
1. Original Rotamer paper
2. New rotamer paper

# Recommended Resources
1. Rotamer Library Homepage: http://dunbrack.fccc.edu/bbdep2010/
2. Rotamer Library Interactive Overview: http://dunbrack.fccc.edu/bbdep2010/ImagesMovies.php

**Only run one of the next two cells.** If you are using Sergey's "wheel install," then you should run the second one; if you're using the "minsizerel" install, use the first one.

In [ ]:
# Notebook setup
import sys
if 'google.colab' in sys.modules:
    !pip install pyrosettacolabsetup
    import pyrosettacolabsetup
    pyrosettacolabsetup.setup()
    print ("Notebook is set for PyRosetta use in Colab.  Have fun!")


Drive already mounted at /content/google_drive; to attempt to forcibly remount, call drive.mount("/content/google_drive", force_remount=True).
Notebook is set for PyRosetta use in Colab.  Have fun!
Notebook is set for PyRosetta use in Colab.  Have fun!


In [ ]:
# Wheel install setup. This takes ~1.5 minutes each time you start a new notebook or do a "factory reset".

# Mounting Google Drive and add it to Python sys path  

google_drive_mount_point = '/content/google_drive'

import os, sys, time

if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount(google_drive_mount_point)

if not os.getenv("DEBUG"):
    google_drive = google_drive_mount_point + '/My Drive'

if not os.getenv("DEBUG"):
    # installing PyRosetta
    if sys.version_info.major != 3 or sys.version_info.minor != 7:
        print('Need Python-3.7 to run!')
        sys.exit(1)

    # upload PyRosetta Linux WHEEL package into your google drive and put it into /PyRosetta dir
    # or alternatively you can download PyRosetta directly from GrayLab web site (but this might take some time!)
    #!mkdir $notebook_path/PyRosetta
    #!cd $notebook_path/PyRosetta && wget --user USERNAME --password PASSWORD https://graylab.jhu.edu/download/PyRosetta4/archive/release/PyRosetta4.Release.python37.ubuntu.wheel/latest.html   

    pyrosetta_distr_path = google_drive + '/PyRosetta' 
    
    # finding path to wheel package, if multiple packages is found take first one
    # replace this with `wheel_path = pyrosetta_distr_path + /<wheel-file-name>.whl` if you want to use particular whl file
    wheel_path = pyrosetta_distr_path + '/' + [ f for f in os.listdir(pyrosetta_distr_path) if f.endswith('.whl')][0]
    
    print(f'Using PyRosetta wheel package: {wheel_path}')

    !pip3 install '{wheel_path}'

Drive already mounted at /content/google_drive; to attempt to forcibly remount, call drive.mount("/content/google_drive", force_remount=True).
Using PyRosetta wheel package: /content/google_drive/My Drive/PyRosetta/pyrosetta-2020.50.post0.dev0+970.commits.3700df14560-cp37-cp37m-linux_x86_64.whl


*Warning*: This notebook uses `pyrosetta.distributed.viewer` code, which runs in `jupyter notebook` and might not run if you're using `jupyterlab`.

Now, the next couple of steps are a little confusing. First, you're going to have to `pip install` the `py3Dmol` package. (You have to pip install it every time you intend to use the py3Dmol viewer.) Next, you have to "restart runtime" (Select the "Runtime" menu at the top, and then select "Restart Runtime" -- caution, do **not** select "Factory reset runtime"). Finally, you have to _rerun_ whichever cell at the top of the notebook that you first ran. If you spent a minute and a half running the wheel install above, fear not: it is nearly instantaneous the second time you run that cell.

In [ ]:
#step 1: pip install py3Dmol
!pip install py3Dmol --user

Step 2: restart runtime

Step 3: re-run either the minsizerel install or the wheel install cells from the top of the notebook.

OK. After those three steps, we're ready to use the viewer; we just won't use it right away.

In [ ]:
import logging
logging.basicConfig(level=logging.INFO)
import pyrosetta
import pyrosetta.toolbox

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

Begin by moving to the directory with your pdb files and loading cetuximab from 1YY8.clean.pdb used in previous workshops.

```
cd google_drive/My\ Drive/pyrbc_202103_notebooks/
pose = pose_from_pdb("inputs/1YY8.clean.pdb")
start_pose = Pose()
start_pose.assign(pose)
```

In [ ]:
#put your CD command here
%cd google_drive/My\ Drive/temp_pyrbc_202103_notebooks/

/content/google_drive/My Drive/temp_pyrbc_202103_notebooks


In [ ]:
from pyrosetta import *
from pyrosetta.teaching import *
init()

INFO:pyrosetta.rosetta:Found rosetta database at: /usr/local/lib/python3.7/dist-packages/pyrosetta/database; using it....
INFO:pyrosetta.rosetta:PyRosetta-4 2021 [Rosetta PyRosetta4.MinSizeRel.python37.ubuntu 2020.50.post.dev+970.commits.3700df145608444753aabbec9c4681ec9b21f74b 2021-02-24T13:24:53] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
INFO:rosetta:core.init: Checking for fconfig files in pwd and ./rosetta/flags
INFO:rosetta:core.init: Rosetta version: PyRosetta4.MinSizeRel.python37.ubuntu r17132 2020.50.post.dev+970.commits.3700df14560 3700df145608444753aabbec9c4681ec9b21f74b http://www.pyrosetta.org 2021-02-24T13:24:53
INFO:rosetta:core.init: command: PyRosetta -ex1 -ex2aro -database /usr/local/lib/python3.7/dist-packages/pyrosetta/database
INFO:rosetta:basic.random.init_random_generator: 'RNG device' seed mode, using '/dev/urandom', seed=-1228256242 seed_offset=0 real_seed=-1228

PyRosetta-4 2021 [Rosetta PyRosetta4.MinSizeRel.python37.ubuntu 2020.50.post.dev+970.commits.3700df145608444753aabbec9c4681ec9b21f74b 2021-02-24T13:24:53] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.


In [ ]:
pose = pose_from_pdb("inputs/1YY8.clean.pdb")
start_pose = Pose()
start_pose.assign(pose)

INFO:rosetta:core.chemical.GlobalResidueTypeSet: Finished initializing fa_standard residue type set.  Created 985 residue types
INFO:rosetta:core.chemical.GlobalResidueTypeSet: Total time to initialize 1.07859 seconds.
INFO:rosetta:core.import_pose.import_pose: File 'inputs/1YY8.clean.pdb' automatically determined to be of type PDB
INFO:rosetta:core.conformation.Conformation: [ WARNING ] missing heavyatom:  CG  on residue ARG 18
INFO:rosetta:core.conformation.Conformation: [ WARNING ] missing heavyatom:  CD  on residue ARG 18
INFO:rosetta:core.conformation.Conformation: [ WARNING ] missing heavyatom:  NE  on residue ARG 18
INFO:rosetta:core.conformation.Conformation: [ WARNING ] missing heavyatom:  CZ  on residue ARG 18
INFO:rosetta:core.conformation.Conformation: [ WARNING ] missing heavyatom:  NH1 on residue ARG 18
INFO:rosetta:core.conformation.Conformation: [ WARNING ] missing heavyatom:  NH2 on residue ARG 18
INFO:rosetta:core.conformation.Conformation: [ WARNING ] missing heavyat

__Question:__ What are the φ, ψ, and χ angles of residue K49?

In [ ]:
print(pose.residue(49).name())
print("Phi: %.5f\nPsi: %.5f\n" %(pose.phi(49), pose.psi(49)))
print("Chi 1: %.5f\nChi 2: %.5f\nChi 3: %.5f\nChi 4: %.5f" %(pose.chi(1, 49), pose.chi(2, 49), pose.chi(3, 49), pose.chi(4, 49)))

LYS
Phi: -122.41752
Psi: 153.08267

Chi 1: 68.12458
Chi 2: -169.18576
Chi 3: -175.37028
Chi 4: -169.58806


Score your pose with the standard full-atom score function. What is the energy of K49? Note the Dunbrack energy component (`fa_dun`), which represents the side-chain conformational probability given phi/psi (Ie backbone-dependant). 

Any energy can be converted to a probability.  Use this energy (E) to calculate the approximate probability of the rotamer (p): p=e^(-E)
```
scorefxn = get_fa_scorefxn()
scorefxn(pose)
energies = pose.energies()
print(energies.residue_total_energies(49))
print(energies.residue_total_energies(49)[pyrosetta.rosetta.core.scoring.fa_dun])
```

In [ ]:
scorefxn = get_fa_scorefxn()
scorefxn(pose)
energies = pose.energies()
print(energies.residue_total_energies(49))
print(energies.residue_total_energies(49)[pyrosetta.rosetta.core.scoring.fa_dun])

INFO:rosetta:core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015


( fa_atr; -8.83354) ( fa_rep; 2.78019) ( fa_sol; 5.92847) ( fa_intra_atr; -1.68168) ( fa_intra_rep; 2.47533) ( fa_intra_sol; 1.20218) ( fa_intra_atr_xover4; -0.52776) ( fa_intra_rep_xover4; 0.191981) ( fa_intra_sol_xover4; 0.292855) ( fa_intra_atr_nonprotein; 0) ( fa_intra_rep_nonprotein; 0) ( fa_intra_sol_nonprotein; 0) ( fa_intra_RNA_base_phos_atr; 0) ( fa_intra_RNA_base_phos_rep; 0) ( fa_intra_RNA_base_phos_sol; 0) ( fa_atr_dummy; 0) ( fa_rep_dummy; 0) ( fa_sol_dummy; 0) ( fa_vdw_tinker; 0) ( lk_hack; 0) ( lk_ball; 0) ( lk_ball_wtd; 0.321463) ( lk_ball_iso; 0) ( lk_ball_bridge; 0) ( lk_ball_bridge_uncpl; 0) ( coarse_fa_atr; 0) ( coarse_fa_rep; 0) ( coarse_fa_sol; 0) ( coarse_beadlj; 0) ( mm_lj_intra_rep; 0) ( mm_lj_intra_atr; 0) ( mm_lj_inter_rep; 0) ( mm_lj_inter_atr; 0) ( mm_twist; 0) ( mm_bend; 0) ( mm_stretch; 0) ( lk_costheta; 0) ( lk_polar; 0) ( lk_nonpolar; 0) ( lk_polar_intra_RNA; 0) ( lk_nonpolar_intra_RNA; 0) ( fa_elec; -2.75782) ( fa_elec_bb_bb; 0) ( fa_elec_bb_sc; 0) ( f

Use `pose.set_chi(<i>, <res_num>, <chi>)` to set the side chain of residue 49 to the all-anti conformation. (Here, `i` is the χ index, and `chi` is the new torsion angle in degrees.) Re-score the pose and note the Dunbrack energy.

In [ ]:
for i in range(1, 5):
    pose.set_chi(i, 49, 180)
    
scorefxn = get_fa_scorefxn()
scorefxn(pose)
energies = pose.energies()
print(energies.residue_total_energies(49))
print(energies.residue_total_energies(49)[pyrosetta.rosetta.core.scoring.fa_dun])

INFO:rosetta:core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015


( fa_atr; -11.6375) ( fa_rep; 789.493) ( fa_sol; 11.6773) ( fa_intra_atr; -1.44441) ( fa_intra_rep; 1.80557) ( fa_intra_sol; 0.785201) ( fa_intra_atr_xover4; -0.343391) ( fa_intra_rep_xover4; 0.112709) ( fa_intra_sol_xover4; 0.105435) ( fa_intra_atr_nonprotein; 0) ( fa_intra_rep_nonprotein; 0) ( fa_intra_sol_nonprotein; 0) ( fa_intra_RNA_base_phos_atr; 0) ( fa_intra_RNA_base_phos_rep; 0) ( fa_intra_RNA_base_phos_sol; 0) ( fa_atr_dummy; 0) ( fa_rep_dummy; 0) ( fa_sol_dummy; 0) ( fa_vdw_tinker; 0) ( lk_hack; 0) ( lk_ball; 0) ( lk_ball_wtd; 0.302965) ( lk_ball_iso; 0) ( lk_ball_bridge; 0) ( lk_ball_bridge_uncpl; 0) ( coarse_fa_atr; 0) ( coarse_fa_rep; 0) ( coarse_fa_sol; 0) ( coarse_beadlj; 0) ( mm_lj_intra_rep; 0) ( mm_lj_intra_atr; 0) ( mm_lj_inter_rep; 0) ( mm_lj_inter_atr; 0) ( mm_twist; 0) ( mm_bend; 0) ( mm_stretch; 0) ( lk_costheta; 0) ( lk_polar; 0) ( lk_nonpolar; 0) ( lk_polar_intra_RNA; 0) ( lk_nonpolar_intra_RNA; 0) ( fa_elec; -9.02391) ( fa_elec_bb_bb; 0) ( fa_elec_bb_sc; 0) (

# Protein Design with a Resfile and FastRelax

Keywords: FastDesign, FastRelax, ResfileCommandOperation, Resfile, ResidueSelector, MoveMapFactory, TaskFactory, TaskOperation, NoRepackDisulfides, IncludeCurrent, ReadResfile, conf2pdb_chain(), pose_from_rcsb(), create_score_function(), CA_rmsd()

## Overview

In this Workshop, we will learn the classic way to design proteins, but in the same breath introduce the concept of design using a flexible backbone protocol.  

This protocol, is essentially design during FastRelax.  A separate class, FastDesign, has a bit more options for design, but essentially, they are the same.  

Many modern designs have used this FastDesign/RelaxedDesign protocol - including many Science papers from the Baker lab and the RosettaAntibodyDesign (RAbD) protocol that we will cover in another tutorial. 

Before this workshop, you should read about the resfile syntax here: https://www.rosettacommons.org/docs/latest/rosetta_basics/file_types/resfiles

### Initialize PyRosetta

In [ ]:
pyrosetta.init("-ignore_unrecognized_res 1 -ex1 -ex2aro -detect_disulf 0")

INFO:pyrosetta.rosetta:Found rosetta database at: /usr/local/lib/python3.7/dist-packages/pyrosetta/database; using it....
INFO:pyrosetta.rosetta:PyRosetta-4 2021 [Rosetta PyRosetta4.MinSizeRel.python37.ubuntu 2020.50.post.dev+970.commits.3700df145608444753aabbec9c4681ec9b21f74b 2021-02-24T13:24:53] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
INFO:rosetta:core.init: Checking for fconfig files in pwd and ./rosetta/flags
INFO:rosetta:core.init: Rosetta version: PyRosetta4.MinSizeRel.python37.ubuntu r17132 2020.50.post.dev+970.commits.3700df14560 3700df145608444753aabbec9c4681ec9b21f74b http://www.pyrosetta.org 2021-02-24T13:24:53


PyRosetta-4 2021 [Rosetta PyRosetta4.MinSizeRel.python37.ubuntu 2020.50.post.dev+970.commits.3700df145608444753aabbec9c4681ec9b21f74b 2021-02-24T13:24:53] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.


INFO:rosetta:core.init: command: PyRosetta -ignore_unrecognized_res 1 -ex1 -ex2aro -detect_disulf 0 -database /usr/local/lib/python3.7/dist-packages/pyrosetta/database
INFO:rosetta:basic.random.init_random_generator: 'RNG device' seed mode, using '/dev/urandom', seed=-1778875097 seed_offset=0 real_seed=-1778875097
INFO:rosetta:basic.random.init_random_generator: RandomGenerator:init: Normal mode, seed=-1778875097 RG_type=mt19937


 For this tutorial, let's use the well-studied native protein crambin from PDB ID 1AB1 (http://www.rcsb.org/structure/1AB1).
 
 Setup the input pose and scorefunction:

In [ ]:
start_pose = pyrosetta.toolbox.rcsb.pose_from_rcsb("1AB1", ATOM=True, CRYS=False)
pose = start_pose.clone()
scorefxn = pyrosetta.create_score_function("ref2015_cart.wts")

INFO:rosetta:core.import_pose.import_pose: File '1AB1.clean.pdb' automatically determined to be of type PDB
INFO:rosetta:core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 3 atoms at position 1 in file 1AB1.clean.pdb. Best match rsd_type:  THR:NtermProteinFull
INFO:rosetta:core.energy_methods.CartesianBondedEnergy: Initializing IdealParametersDatabase with default Ks=300 , 80 , 80 , 10 , 80
INFO:rosetta:basic.io.database: Database file opened: scoring/score_functions/bondlength_bondangle/default-lengths.txt
INFO:rosetta:core.energy_methods.CartesianBondedEnergy: Read 759 bb-independent lengths.
INFO:rosetta:basic.io.database: Database file opened: scoring/score_functions/bondlength_bondangle/default-angles.txt
INFO:rosetta:core.energy_methods.CartesianBondedEnergy: Read 1434 bb-independent angles.
INFO:rosetta:basic.io.database: Database file opened: scoring/score_functions/bondlength_bondangle/default-torsions.txt
INFO:rosetta:core.energy_methods.CartesianBondedEnergy: R

 Make of list of which residues are cysteine:

In [ ]:
cys_res = []
for i, aa in enumerate(start_pose.sequence(), start=1):
    if aa == "C":
        cys_res.append(i)
print(cys_res)

[3, 4, 16, 26, 32, 40]


Inspect `start_pose` using the `PyMolMover` or `dump_pdb()`

## Design strategy:

Design away the cysteine residues (i.e. disulfide bonds) using a resfile, allowing all side-chains to re-pack and all backbone and side-chain torsions to minimize using the `FastRelax` mover.

 Read more about resfile file structure at https://www.rosettacommons.org/docs/latest/rosetta_basics/file_types/resfiles

To write a resfile, we need to know which chain to mutate.

We can see that the pose consists of only chain "A" by printing the `pose.pdb_info()` object:

In [ ]:
print(pose.pdb_info())

PDB file name: 1AB1.clean.pdb
 Pose Range  Chain    PDB Range  |   #Residues         #Atoms

0001 -- 0046    A 0001  -- 0046  |   0046 residues;    00649 atoms
                           TOTAL |   0046 residues;    00649 atoms



 More programmatically, we could find which chains are in the `pose` using `pyrosetta.rosetta.core.pose.conf2pdb_chain(pose)` which returns a `pyrosetta.rosetta.std.map_unsigned_long_char` object which is iterable.

In [ ]:
print(pyrosetta.rosetta.core.pose.conf2pdb_chain(pose))

map_unsigned_long_char{1: A}


In [ ]:
for k, v in pyrosetta.rosetta.core.pose.conf2pdb_chain(pose).items():
    print(v)

A


 So we could write a resfile to disc indicating design specifications to mutate only the cysteine residues in chain "A".  Use the syntax described below, and save your resfile in this directory as `redes_cys.resfile`.
 
 https://www.rosettacommons.org/docs/latest/rosetta_basics/file_types/resfiles

First, create a directory in your pyrbc_202103_notebooks directory named "outputs," and then write your resfile to that directory

In [ ]:
resfile = "./outputs/redes_cys.resfile"  # this is the file you should write to; we will read from this file later

with open(resfile, "w") as f:
  f.write("NATAA\n")
  f.write("start\n")

  for i in cys_res:
    f.write("{0} {1} ALLAAxc\n".format(i, pyrosetta.rosetta.core.pose.conf2pdb_chain(pose)[1]))

!cat {resfile}

NATAA
start
3 A ALLAAxc
4 A ALLAAxc
16 A ALLAAxc
26 A ALLAAxc
32 A ALLAAxc
40 A ALLAAxc


Note that we don't necessarily need a resfile to use resfile commands.  We can now do this in an intuitive way through code and `ResidueSelectors` using the `ResfileCommandOperation`.  The main docs for the XML interface are available below, however the code-level interface is extremely similar.  Use the ? to get more info on this.  The operation is located in `pyrosetta.rosetta.core.pack.task.operation` as we saw this location in the previous tutorial.

https://www.rosettacommons.org/docs/latest/scripting_documentation/RosettaScripts/TaskOperations/taskoperations_pages/ResfileCommandOperation

 Now we can setup the TaskOperations for the `FastRelax` mover. These tell `FastRelax` which residues to design or repack during the packer steps in `FastRelax`.  You should be familiar with this from the previous tutorial
 
We use the `IncludeCurrent` to include the current rotamer of from the crystal structure during packing.

In [ ]:
# The task factory accepts all the task operations
tf = pyrosetta.rosetta.core.pack.task.TaskFactory()

# These are pretty standard
tf.push_back(pyrosetta.rosetta.core.pack.task.operation.InitializeFromCommandline())
tf.push_back(pyrosetta.rosetta.core.pack.task.operation.IncludeCurrent())
tf.push_back(pyrosetta.rosetta.core.pack.task.operation.NoRepackDisulfides())

# Include the resfile
tf.push_back(pyrosetta.rosetta.core.pack.task.operation.ReadResfile(resfile))

# Convert the task factory into a PackerTask to take a look at it
packer_task = tf.create_task_and_apply_taskoperations(pose)
# View the PackerTask
print(packer_task)

INFO:rosetta:core.pack.task: Packer task: initialize from command line()


#Packer_Task

Threads to request: ALL AVAILABLE

resid	pack?	design?	allowed_aas
1	TRUE	FALSE	THR:NtermProteinFull
2	TRUE	FALSE	THR
3	TRUE	TRUE	ALA,ASP,GLU,PHE,GLY,HIS,HIS_D,ILE,LYS,LEU,MET,ASN,PRO,GLN,ARG,SER,THR,VAL,TRP,TYR
4	TRUE	TRUE	ALA,ASP,GLU,PHE,GLY,HIS,HIS_D,ILE,LYS,LEU,MET,ASN,PRO,GLN,ARG,SER,THR,VAL,TRP,TYR
5	TRUE	FALSE	PRO
6	TRUE	FALSE	SER
7	TRUE	FALSE	ILE
8	TRUE	FALSE	VAL
9	TRUE	FALSE	ALA
10	TRUE	FALSE	ARG
11	TRUE	FALSE	SER
12	TRUE	FALSE	ASN
13	TRUE	FALSE	PHE
14	TRUE	FALSE	ASN
15	TRUE	FALSE	VAL
16	TRUE	TRUE	ALA,ASP,GLU,PHE,GLY,HIS,HIS_D,ILE,LYS,LEU,MET,ASN,PRO,GLN,ARG,SER,THR,VAL,TRP,TYR
17	TRUE	FALSE	ARG
18	TRUE	FALSE	LEU
19	TRUE	FALSE	PRO
20	TRUE	FALSE	GLY
21	TRUE	FALSE	THR
22	TRUE	FALSE	SER
23	TRUE	FALSE	GLU
24	TRUE	FALSE	ALA
25	TRUE	FALSE	ILE
26	TRUE	TRUE	ALA,ASP,GLU,PHE,GLY,HIS,HIS_D,ILE,LYS,LEU,MET,ASN,PRO,GLN,ARG,SER,THR,VAL,TRP,TYR
27	TRUE	FALSE	ALA
28	TRUE	FALSE	THR
29	TRUE	FALSE	TYR
30	TRUE	FALSE	THR
31	TRUE	FALSE	GLY
32	TRUE	TRUE	ALA,ASP,GLU,PHE,GLY,HIS,HIS_D,IL

 The PackerTask looks as intended! 

 Now we can set up a `MoveMap` or a `MoveMapFactory` to specify which torsions are free to minimize during the minimization steps of the `FastDesign` mover

In [ ]:
# Set up a MoveMapFactory
mmf = pyrosetta.rosetta.core.select.movemap.MoveMapFactory()
mmf.all_bb(setting=True)
mmf.all_bondangles(setting=True)
mmf.all_bondlengths(setting=True)
mmf.all_chi(setting=True)
mmf.all_jumps(setting=True)
mmf.set_cartesian(setting=True)

In [ ]:
# Set up a MoveMap
# mm = pyrosetta.rosetta.core.kinematics.MoveMap()
# mm.set_bb(True)
# mm.set_chi(True)
# mm.set_jump(True)

# If needed, you could turn off bb and chi torsions for individual residues like this:

# vector1 of true/false for each residue in the pose
# subset_to_minimize = do_something_set.apply(pose)

# for i in range(1, pose.size() + 1):
#     if (not subset_to_minimize[i]):
#         mm.set_bb(i, False)
#         mm.set_chi(i, False)

 Because some Movers only take as input a `MoveMap`, for backwards-compatibility one could generate a `MoveMap` from a `MoveMapFactory` using the `MoveMapFactory` function `create_movemap_from_pose(pose)`

 Now let's double-check some more `pose` information to verify that we are ready for `FastRelax`:

In [ ]:
display_pose = pyrosetta.rosetta.protocols.fold_from_loops.movers.DisplayPoseLabelsMover()
display_pose.tasks(tf)
display_pose.movemap_factory(mmf)
display_pose.apply(pose)

INFO:rosetta:protocols.DsspMover: LEELLLHHHHHHHHHHHLLLLLHHHHHHHHLLEELLLLLLLLLLLL
INFO:rosetta:protocols.fold_from_loops.DisplayPoseLabelsMover: SEQUENCE       TTCCPSIVARSNFNVCRLPGTSEAICATYTGCIIIPGATCPGDYAN
INFO:rosetta:protocols.fold_from_loops.DisplayPoseLabelsMover: STRUCTURE      LEELLLHHHHHHHHHHHLLLLLHHHHHHHHLLEELLLLLLLLLLLL
INFO:rosetta:protocols.fold_from_loops.DisplayPoseLabelsMover: MVMP_BB        **********************************************
INFO:rosetta:protocols.fold_from_loops.DisplayPoseLabelsMover: MVMP_CHI       **********************************************
INFO:rosetta:core.pack.task: Packer task: initialize from command line()
INFO:rosetta:protocols.fold_from_loops.DisplayPoseLabelsMover: DESIGN         --**-----------*---------*-----*-------*------
INFO:rosetta:protocols.fold_from_loops.DisplayPoseLabelsMover: REPACK         **********************************************
INFO:rosetta:protocols.fold_from_loops.DisplayPoseLabelsMover: FOLDTREE       ******************

 Setting up `FastRelax` prints the default `relaxscript`, showing the `ramp_repack_min` settings with the following assignments:
>ramp_repack_min [scale:fa_rep] [min_tolerance] [coord_cst_weight]

In [ ]:
fr = pyrosetta.rosetta.protocols.relax.FastRelax(scorefxn_in=scorefxn, standard_repeats=1)
fr.cartesian(True)
fr.set_task_factory(tf)
fr.set_movemap_factory(mmf)
fr.min_type("lbfgs_armijo_nonmonotone") # For non-Cartesian scorefunctions, use "dfpmin_armijo_nonmonotone"

#Note that this min_type is automatically set when you set the cartesian option. 
#  But it is good to be aware of this - as not all protocols will do this for you.

#fr.set_movemap(mm) # Could have optionally specified a MoveMap instead of MoveMapFactory
#fr.minimize_bond_angles(True) # If not using MoveMapFactory, could specify bond angle minimization here
#fr.minimize_bond_lengths(True) # If not using MoveMapFactory, could specify bond length minimization here

INFO:rosetta:protocols.relax.RelaxScriptManager: Reading relax scripts list from database.
INFO:rosetta:protocols.relax.RelaxScriptManager: Looking for MonomerRelax2019.txt
INFO:rosetta:protocols.relax.RelaxScriptManager: ================== Reading script file: /usr/local/lib/python3.7/dist-packages/pyrosetta/database/sampling/relax_scripts/MonomerRelax2019.txt ==================
INFO:rosetta:protocols.relax.RelaxScriptManager: repeat %%nrepeats%%
INFO:rosetta:protocols.relax.RelaxScriptManager: coord_cst_weight 1.0
INFO:rosetta:protocols.relax.RelaxScriptManager: scale:fa_rep 0.040
INFO:rosetta:protocols.relax.RelaxScriptManager: repack
INFO:rosetta:protocols.relax.RelaxScriptManager: scale:fa_rep 0.051
INFO:rosetta:protocols.relax.RelaxScriptManager: min 0.01
INFO:rosetta:protocols.relax.RelaxScriptManager: coord_cst_weight 0.5
INFO:rosetta:protocols.relax.RelaxScriptManager: scale:fa_rep 0.265
INFO:rosetta:protocols.relax.RelaxScriptManager: repack
INFO:rosetta:protocols.relax.Relax

For recommendations on setting `fr.min_type()` for the scorefunction being used, see: https://www.rosettacommons.org/docs/latest/rosetta_basics/structural_concepts/minimization-overview#recommendations

 Run Fast(Design)! Note: this takes ~1min 31s

In [ ]:
fr.apply(pose)
pose.dump_pdb("outputs/1ab1_resfile_relaxed.pdb")

INFO:rosetta:core.energy_methods.CartesianBondedEnergy: Creating new peptide-bonded energy container (46)
INFO:rosetta:protocols.relax.FastRelax: CMD: repeat  1274.78  0  0  0.55
INFO:rosetta:protocols.relax.FastRelax: CMD: coord_cst_weight  1274.78  0  0  0.55
INFO:rosetta:protocols.relax.FastRelax: CMD: scale:fa_rep  406.204  0  0  0.022
INFO:rosetta:core.pack.task: Packer task: initialize from command line()
INFO:rosetta:core.pack.pack_rotamers: built 5671 rotamers at 46 positions.
INFO:rosetta:core.pack.interaction_graph.interaction_graph_factory: Instantiating PDInteractionGraph
INFO:rosetta:protocols.relax.FastRelax: CMD: repack  75.6883  0  0  0.022
INFO:rosetta:protocols.relax.FastRelax: CMD: scale:fa_rep  83.4309  0  0  0.02805
INFO:rosetta:protocols.relax.FastRelax: CMD: min  -176.221  0.716057  0.716057  0.02805
INFO:rosetta:protocols.relax.FastRelax: CMD: coord_cst_weight  -176.221  0.716057  0.716057  0.02805
INFO:rosetta:protocols.relax.FastRelax: CMD: scale:fa_rep  -72.4

True

### Analysis

Inspect the resulting design!

 By how many Angstroms RMSD did the backbone Cα atoms move?

In [ ]:
pyrosetta.rosetta.core.scoring.CA_rmsd(start_pose, pose)

0.5374792814254761

What is the delta `total_score` from `start_pose` to `pose`? Why is it large?

In [ ]:
delta_total_score = scorefxn(pose) - scorefxn(start_pose)
print(delta_total_score)

INFO:rosetta:core.energy_methods.CartesianBondedEnergy: Creating new peptide-bonded energy container (46)


-1379.15070574072


 What is the per-residue energy difference for each mutated position between `start_pose` and `pose`?

In [ ]:
for i in cys_res:
  print("Residue #: ", i)
  print("Relaxed pose: ", pyrosetta.rosetta.protocols.relax.get_per_residue_scores(pose, pyrosetta.rosetta.core.scoring.ScoreType.total_score)[i - 1])
  print("Start pose: ", pyrosetta.rosetta.protocols.relax.get_per_residue_scores(start_pose, pyrosetta.rosetta.core.scoring.ScoreType.total_score)[i - 1])

Residue #:  3
Relaxed pose:  -2.198366153044414
Start pose:  9.949274702677952
Residue #:  4
Relaxed pose:  -2.6002366356597775
Start pose:  161.06299751955535
Residue #:  16
Relaxed pose:  0.04939967402494855
Start pose:  35.16917408036165
Residue #:  26
Relaxed pose:  -1.4184857949233842
Start pose:  6.059158875174292
Residue #:  32
Relaxed pose:  -1.304546479338188
Start pose:  5.015792828193076
Residue #:  40
Relaxed pose:  2.127620519893319
Start pose:  8.369850772545249


# Protein Design 2

Keywords: FastDesign, FastRelax, ResidueSelector, TaskFactory, TaskOperation, pose_from_rcsb, get_residues_from_subset, ResidueNameSelector, ChainSelector, ResiduePropertySelector, NotResidueSelector, AndResidueSelector, RestrictToRepackingRLT, RestrictAbsentCanonicalAASRLT, NoRepackDisulfides, OperateOnResidueSubset

## Overview

Here, we will expand upon our knowledge of ResidueSelectors and TaskOperations and how to use them.  We can once again use FastRelax or FastDesign to do design.  Since we don't need any additional tools offered by FastDesign, we will use FastRelax.  I like to call it RelaxedDesign. :)

In [ ]:
import logging
logging.basicConfig(level=logging.INFO)
import os
import pyrosetta
import pyrosetta.toolbox
import site

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Initialize PyRosetta 

In [ ]:
pyrosetta.init("-ignore_unrecognized_res 1 -ex1 -ex2aro")

INFO:pyrosetta.rosetta:Found rosetta database at: /usr/local/lib/python3.7/dist-packages/pyrosetta/database; using it....
INFO:pyrosetta.rosetta:PyRosetta-4 2021 [Rosetta PyRosetta4.MinSizeRel.python37.ubuntu 2020.50.post.dev+970.commits.3700df145608444753aabbec9c4681ec9b21f74b 2021-02-24T13:24:53] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
INFO:rosetta:core.init: Checking for fconfig files in pwd and ./rosetta/flags
INFO:rosetta:core.init: Rosetta version: PyRosetta4.MinSizeRel.python37.ubuntu r17132 2020.50.post.dev+970.commits.3700df14560 3700df145608444753aabbec9c4681ec9b21f74b http://www.pyrosetta.org 2021-02-24T13:24:53
INFO:rosetta:core.init: command: PyRosetta -ignore_unrecognized_res 1 -ex1 -ex2aro -database /usr/local/lib/python3.7/dist-packages/pyrosetta/database
INFO:rosetta:basic.random.init_random_generator: 'RNG device' seed mode, using '/dev/urandom', seed=-118602821 see

PyRosetta-4 2021 [Rosetta PyRosetta4.MinSizeRel.python37.ubuntu 2020.50.post.dev+970.commits.3700df145608444753aabbec9c4681ec9b21f74b 2021-02-24T13:24:53] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.


 For this tutorial, let's again use the native protein crambin from PDB ID 1AB1 (http://www.rcsb.org/structure/1AB1)
 Setup the input pose and scorefunction:

In [ ]:
start_pose = pyrosetta.toolbox.rcsb.pose_from_rcsb("1AB1", ATOM=True, CRYS=False)
pose = start_pose.clone()
scorefxn = pyrosetta.create_score_function("ref2015_cart.wts")

INFO:rosetta:core.import_pose.import_pose: File '1AB1.clean.pdb' automatically determined to be of type PDB
INFO:rosetta:core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 3 atoms at position 1 in file 1AB1.clean.pdb. Best match rsd_type:  THR:NtermProteinFull
INFO:rosetta:core.conformation.Conformation: Found disulfide between residues 3 40
INFO:rosetta:core.conformation.Conformation: current variant for 3 CYS
INFO:rosetta:core.conformation.Conformation: current variant for 40 CYS
INFO:rosetta:core.conformation.Conformation: current variant for 3 CYD
INFO:rosetta:core.conformation.Conformation: current variant for 40 CYD
INFO:rosetta:core.conformation.Conformation: Found disulfide between residues 4 32
INFO:rosetta:core.conformation.Conformation: current variant for 4 CYS
INFO:rosetta:core.conformation.Conformation: current variant for 32 CYS
INFO:rosetta:core.conformation.Conformation: current variant for 4 CYD
INFO:rosetta:core.conformation.Conformation: current varian

 To list the cysteine residues in crambin, now we use ResidueSelectors:
 Note difference in residue names "CYS:disulfide" vs. "CYS"
 
 Note that we could also use the `ResiduePropertySelector` and use the `DISULFIDE_BONDED` property to grab these.

In [ ]:
disulfide_res = pyrosetta.rosetta.core.select.residue_selector.ResidueNameSelector("CYS:disulfide")
print(pyrosetta.rosetta.core.select.get_residues_from_subset(disulfide_res.apply(pose)))

vector1_unsigned_long[3, 4, 16, 26, 32, 40]


You could inspect the starting pose using the `PyMolMover` or `dump_pdb()`. However, by default, disulfide bonds are not drawn even if they exist in PyRosetta.

You can also use this shortcut to view a pose in `py3Dmol` which recognizes disulfide bonds in PyRosetta:
 
Copy the file `visualization.py` from the Session12 directory into the pyrbc_202103_notebooks directory. Next, run the `visualization.py` script, which will define a function `view_pose`:

````
%run ./visualization.py
```

In [ ]:
%run ./Sessions/Session12/visualization.py

Now that `view_pose` has been defined, run it!
```
view_pose(pose)
```

In [ ]:
view_pose(pose)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## Design strategy

Design away the cysteine residues (i.e. disulfide bonds) using ResidueSelectors and TaskOperations, allowing all side-chains to re-pack and all backbone and side-chain torsions to minimize using the `FastDesign` mover.

 Note: because we did not include the `-detect_disulfide 0` command line flag in `pyrosetta.init()`, in order to mutate disulfide bonded cysteine residues using packing steps in the `FastDesign` mover, first we need to break the disulfide bonds:

In [ ]:
for i in pyrosetta.rosetta.core.select.get_residues_from_subset(disulfide_res.apply(pose)):
    for j in pyrosetta.rosetta.core.select.get_residues_from_subset(disulfide_res.apply(pose)):
        if pyrosetta.rosetta.core.conformation.is_disulfide_bond(pose.conformation(), i, j):
            pyrosetta.rosetta.core.conformation.break_disulfide(pose.conformation(), i, j)

 Now we can declare a `ResidueSelector` for cysteine residues:

In [ ]:
cys_res = pyrosetta.rosetta.core.select.residue_selector.ResidueNameSelector("CYS")
print(pyrosetta.rosetta.core.select.get_residues_from_subset(cys_res.apply(pose)))

vector1_unsigned_long[3, 4, 16, 26, 32, 40]


 Note: the pose consists of only chain "A"

In [ ]:
print(pose.pdb_info())

PDB file name: 1AB1.clean.pdb
 Pose Range  Chain    PDB Range  |   #Residues         #Atoms

0001 -- 0046    A 0001  -- 0046  |   0046 residues;    00649 atoms
                           TOTAL |   0046 residues;    00649 atoms



 Although the pose contains only chain A, if the pose contained other chains we could make the selection to only select chain A and cysteine residues using an `AndResidueSelector`. We will use this ResidueSelector with `RestrictAbsentCanonicalAASRLT` (restrict absent canonical amino acids residue level task operation).

In [ ]:
chain_A = pyrosetta.rosetta.core.select.residue_selector.ChainSelector("A")
print(pyrosetta.rosetta.core.select.get_residues_from_subset(chain_A.apply(pose)))
chain_A_cys_res = pyrosetta.rosetta.core.select.residue_selector.AndResidueSelector(selector1=cys_res, selector2=chain_A)
print(pyrosetta.rosetta.core.select.get_residues_from_subset(chain_A_cys_res.apply(pose)))

vector1_unsigned_long[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46]
vector1_unsigned_long[3, 4, 16, 26, 32, 40]


 Specify a `NotResidueSelector` selecting everything except the `chain_A_cys_res` ResidueSelector to use with a `RestrictToRepackingRLT` (restrict to repacking residue level task operation):

In [ ]:
not_chain_A_cys_res = pyrosetta.rosetta.core.select.residue_selector.NotResidueSelector(chain_A_cys_res)
print(pyrosetta.rosetta.core.select.get_residues_from_subset(not_chain_A_cys_res.apply(pose)))

vector1_unsigned_long[1, 2, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 18, 19, 20, 21, 22, 23, 24, 25, 27, 28, 29, 30, 31, 33, 34, 35, 36, 37, 38, 39, 41, 42, 43, 44, 45, 46]


 Quickly view the `not_chain_A_cys_res` ResidueSelector:

 Now we can set up the TaskOperations for `FastDesign`:

In [ ]:
# The task factory accepts all the task operations
tf = pyrosetta.rosetta.core.pack.task.TaskFactory()

# These are pretty standard
tf.push_back(pyrosetta.rosetta.core.pack.task.operation.InitializeFromCommandline())
tf.push_back(pyrosetta.rosetta.core.pack.task.operation.IncludeCurrent())
tf.push_back(pyrosetta.rosetta.core.pack.task.operation.NoRepackDisulfides())


# Disable design (i.e. repack only) on not_chain_A_cys_res
tf.push_back(pyrosetta.rosetta.core.pack.task.operation.OperateOnResidueSubset(
    pyrosetta.rosetta.core.pack.task.operation.RestrictToRepackingRLT(), not_chain_A_cys_res))

# Enable design on chain_A_cys_res
aa_to_design = pyrosetta.rosetta.core.pack.task.operation.RestrictAbsentCanonicalAASRLT()
aa_to_design.aas_to_keep("ADEFGHIKLMNPQRSTVWY")
tf.push_back(pyrosetta.rosetta.core.pack.task.operation.OperateOnResidueSubset(
    aa_to_design, chain_A_cys_res))

# Convert the task factory into a PackerTask
packer_task = tf.create_task_and_apply_taskoperations(pose)
# View the PackerTask
print(packer_task)

INFO:rosetta:core.pack.task: Packer task: initialize from command line()


#Packer_Task

Threads to request: ALL AVAILABLE

resid	pack?	design?	allowed_aas
1	TRUE	FALSE	THR:NtermProteinFull
2	TRUE	FALSE	THR
3	TRUE	TRUE	ALA,ASP,GLU,PHE,GLY,HIS,HIS_D,ILE,LYS,LEU,MET,ASN,PRO,GLN,ARG,SER,THR,VAL,TRP,TYR
4	TRUE	TRUE	ALA,ASP,GLU,PHE,GLY,HIS,HIS_D,ILE,LYS,LEU,MET,ASN,PRO,GLN,ARG,SER,THR,VAL,TRP,TYR
5	TRUE	FALSE	PRO
6	TRUE	FALSE	SER
7	TRUE	FALSE	ILE
8	TRUE	FALSE	VAL
9	TRUE	FALSE	ALA
10	TRUE	FALSE	ARG
11	TRUE	FALSE	SER
12	TRUE	FALSE	ASN
13	TRUE	FALSE	PHE
14	TRUE	FALSE	ASN
15	TRUE	FALSE	VAL
16	TRUE	TRUE	ALA,ASP,GLU,PHE,GLY,HIS,HIS_D,ILE,LYS,LEU,MET,ASN,PRO,GLN,ARG,SER,THR,VAL,TRP,TYR
17	TRUE	FALSE	ARG
18	TRUE	FALSE	LEU
19	TRUE	FALSE	PRO
20	TRUE	FALSE	GLY
21	TRUE	FALSE	THR
22	TRUE	FALSE	SER
23	TRUE	FALSE	GLU
24	TRUE	FALSE	ALA
25	TRUE	FALSE	ILE
26	TRUE	TRUE	ALA,ASP,GLU,PHE,GLY,HIS,HIS_D,ILE,LYS,LEU,MET,ASN,PRO,GLN,ARG,SER,THR,VAL,TRP,TYR
27	TRUE	FALSE	ALA
28	TRUE	FALSE	THR
29	TRUE	FALSE	TYR
30	TRUE	FALSE	THR
31	TRUE	FALSE	GLY
32	TRUE	TRUE	ALA,ASP,GLU,PHE,GLY,HIS,HIS_D,IL

 This time we will set up a `MoveMap` to establish which torsions are free to minimize

In [ ]:
mm = pyrosetta.rosetta.core.kinematics.MoveMap()
mm.set_bb(True)
mm.set_chi(True)
mm.set_jump(True)

Set up `FastDesign`

In [ ]:
rel_design = pyrosetta.rosetta.protocols.relax.FastRelax(scorefxn_in=scorefxn, standard_repeats=1, script_file="MonomerDesign2019")
rel_design.cartesian(True)
rel_design.set_task_factory(tf)
rel_design.set_movemap(mm)
rel_design.minimize_bond_angles(True)
rel_design.minimize_bond_lengths(True)

INFO:rosetta:protocols.relax.RelaxScriptManager: Looking for MonomerDesign2019.txt
INFO:rosetta:protocols.relax.RelaxScriptManager: ================== Reading script file: /usr/local/lib/python3.7/dist-packages/pyrosetta/database/sampling/relax_scripts/MonomerDesign2019.txt ==================
INFO:rosetta:protocols.relax.RelaxScriptManager: repeat %%nrepeats%%
INFO:rosetta:protocols.relax.RelaxScriptManager: coord_cst_weight 1.0
INFO:rosetta:protocols.relax.RelaxScriptManager: scale:fa_rep 0.059
INFO:rosetta:protocols.relax.RelaxScriptManager: repack
INFO:rosetta:protocols.relax.RelaxScriptManager: scale:fa_rep 0.092
INFO:rosetta:protocols.relax.RelaxScriptManager: min 0.01
INFO:rosetta:protocols.relax.RelaxScriptManager: coord_cst_weight 0.5
INFO:rosetta:protocols.relax.RelaxScriptManager: scale:fa_rep 0.280
INFO:rosetta:protocols.relax.RelaxScriptManager: repack
INFO:rosetta:protocols.relax.RelaxScriptManager: scale:fa_rep 0.323
INFO:rosetta:protocols.relax.RelaxScriptManager: min 0.

 Run the `FastDesign` mover. Note: this takes ~39.6 s

In [ ]:
rel_design.apply(pose)
pose.dump_pdb("outputs/1ab1_res_selector_re1.pdb")

INFO:rosetta:core.energy_methods.CartesianBondedEnergy: Creating new peptide-bonded energy container (46)
INFO:rosetta:protocols.relax.FastRelax: CMD: repeat  1274.78  0  0  0.55
INFO:rosetta:protocols.relax.FastRelax: CMD: coord_cst_weight  1274.78  0  0  0.55
INFO:rosetta:protocols.relax.FastRelax: CMD: scale:fa_rep  423.395  0  0  0.03245
INFO:rosetta:core.pack.task: Packer task: initialize from command line()
INFO:rosetta:core.pack.pack_rotamers: built 5220 rotamers at 46 positions.
INFO:rosetta:core.pack.interaction_graph.interaction_graph_factory: Instantiating PDInteractionGraph
INFO:rosetta:protocols.relax.FastRelax: CMD: repack  80.8061  0  0  0.03245
INFO:rosetta:protocols.relax.FastRelax: CMD: scale:fa_rep  84.7937  0  0  0.0506
INFO:rosetta:protocols.relax.FastRelax: CMD: min  -159.089  1.05967  1.05967  0.0506
INFO:rosetta:protocols.relax.FastRelax: CMD: coord_cst_weight  -159.089  1.05967  1.05967  0.0506
INFO:rosetta:protocols.relax.FastRelax: CMD: scale:fa_rep  -114.304

True

 Inspect the resulting design!

In [ ]:
view_pose(pose)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## *Design Challenge*:

### Copy and modify this script to:
 - Repack all residues and allow all degrees of freedom to minimize (i.e. FastRelax) `start_pose` prior to downstream design. Use the resulting `start_pose` for analysis.
 - Re-design the cysteine residues in `start_pose` to allow only hydrophobic residues excluding glycine and proline
 - Only re-pack residues within a 6 Angstroms radius around the neighborhood of each cysteine residue (Cα-Cα distance)
   - Note this will require using:
   - The NeighborhoodResidueSelector: `pyrosetta.rosetta.core.select.residue_selector.NeighborhoodResidueSelector(selector, distance, include_focus_in_subset)`
   - The prevent repacking residue level task operation: `pyrosetta.rosetta.core.pack.task.operation.PreventRepackingRLT()`
 - Minimize only the side-chains (chi degrees of freedom) being re-packed or designed
 - Allow the backbone torsions to minimize, except for the first, last and any proline phi/psi angles
 
## *Extra Challenge!*

 Instead of using `pyrosetta.rosetta.core.pack.task.operation.OperateOnResidueSubset` TaskOperations, use `pyrosetta.rosetta.protocols.task_operations.ResfileCommandOperation` TaskOperations by applying resfile-style commands to ResidueSelectors. Example:
 > repack_only_task_operation = pyrosetta.rosetta.protocols.task_operations.ResfileCommandOperation()   
 > repack_only_task_operation.set_command("POLAR EMPTY NC R2 NC T6 NC OP5")   
 > repack_only_task_operation.set_residue_selector(selector)   
 > tf.push_back(repack_only_task_operation)

 By how many Angstroms RMSD did the backbone Cα atoms move?

In [ ]:
pyrosetta.rosetta.core.scoring.CA_rmsd(start_pose, pose)

0.9092081189155579

 What is the delta `total_score` from `start_pose` to `pose`?

In [ ]:
delta_total_score = scorefxn(pose) - scorefxn(start_pose)
print(delta_total_score)

INFO:rosetta:core.energy_methods.CartesianBondedEnergy: Creating new peptide-bonded energy container (46)


-506.33418627927233


 What is the per-residue energy difference for each mutated position between `start_pose` and `pose`?

In [ ]:
# # for loop through cys_res
# ## per-residue energy: pyrosetta.rosetta.protocols.relax.get_per_residue_scores(pose, pyrosetta.rosetta.core.scoring.ScoreType.total_score)
# ## find per residue energy in pose and original pose
# ## subtraction between original pose and pose
# delta_total_score = scorefxn(pose) - scorefxn(start_pose)
# print(delta_total_score)
# for i in cys_res:
#     pose_total_score = pyrosetta.rosetta.protocols.relax.get_per_residue_scores(pose, pyrosetta.rosetta.core.scoring.ScoreType.total_score)[i - 1]
#     start_pose_total_score = pyrosetta.rosetta.protocols.relax.get_per_residue_scores(start_pose, pyrosetta.rosetta.core.scoring.ScoreType.total_score)[i - 1]
#     print("The delta total_score for residue {} between start_pose and pose is {}".format(i, pose_total_score - start_pose_total_score))

-506.33418627927233


TypeError: ignored

# HBNet Before Design

Keywords: HBNet, OperateOnResidueSubset, getPoseExtraScore, InterGroupInterfaceByVectorSelector, ChainSelector, PreventRepackingRLT, RestrictToRepackingRLT, OperateOnResidueSubset, ResiduePDBInfoHasLabelSelector, PackRotamersMover

Sometimes in Rosetta we want to run implicit multistage design. That is, we want to optimize one conformation while implicitly modeling another (either negatively or positively). There are many ways to accomplish this depending on your interests. In this section we will look at HBNet, a tool for explicitly designing hydrogen bond networks.

One negative-design approach is to implicitly model binding specificity. Designing a complicated network of hydrogen bonds at one interface will implicitly destabilize other interfaces. Hydrogen bonds are so sensitive to geometry that competing interfaces are unlikely to be able to "satisfy" the network well enough to remain competetive.

The previous example can also be viewed through the implicit positive-design lens as well. We often find that Rosetta designs very hydrophobic interfaces (especially with newer score functions). Running HBNet before the traditional design protocols can boost the polar residue concentration of your interface in exchange for a small cost packing quality. In other words, we can implicitly stabilize the unbound state by running HBNet, but it might mildly destabilize the bound state.

Our experience shows that it is useful to run both with and without HBNet, depending on your design case. It is possible that the default design protocol handles your implicits states well enough. When that fails, though, there is not much to do to fix it other than to run pre-design protocols like HBNet. An added benefit of HBNet is that it can provide "seeds" for packing, which can influence design diversity if nothing else.

In [ ]:
# From previous section:
from pyrosetta import *
from pyrosetta.teaching import *
pyrosetta.init("-mute core -mute basic")
print( "init complete" )

INFO:pyrosetta.rosetta:Found rosetta database at: /usr/local/lib/python3.7/dist-packages/pyrosetta/database; using it....
INFO:pyrosetta.rosetta:PyRosetta-4 2021 [Rosetta PyRosetta4.MinSizeRel.python37.ubuntu 2020.50.post.dev+970.commits.3700df145608444753aabbec9c4681ec9b21f74b 2021-02-24T13:24:53] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
INFO:rosetta:core.init: Checking for fconfig files in pwd and ./rosetta/flags
INFO:rosetta:core.init: Rosetta version: PyRosetta4.MinSizeRel.python37.ubuntu r17132 2020.50.post.dev+970.commits.3700df14560 3700df145608444753aabbec9c4681ec9b21f74b http://www.pyrosetta.org 2021-02-24T13:24:53
INFO:rosetta:core.init: command: PyRosetta -mute core -mute basic -database /usr/local/lib/python3.7/dist-packages/pyrosetta/database
INFO:rosetta:basic.random.init_random_generator: 'RNG device' seed mode, using '/dev/urandom', seed=-2104333995 seed_offset=0 real_

PyRosetta-4 2021 [Rosetta PyRosetta4.MinSizeRel.python37.ubuntu 2020.50.post.dev+970.commits.3700df145608444753aabbec9c4681ec9b21f74b 2021-02-24T13:24:53] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
init complete


We prepare for HBNet the same way that we prepare for packing. We setup the pose and score function as before...

In [ ]:
pose = pose_from_pdb("inputs/hbnet_example.pdb")
start_pose = Pose()
start_pose.assign(pose)
scorefxn = get_fa_scorefxn()
print( "setup complete" )

INFO:rosetta:core.import_pose.import_pose: File 'inputs/hbnet_example.pdb' automatically determined to be of type PDB
INFO:rosetta:core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] PDB reader is ignoring atom  CD  in residue 131 D.  Pass flag -ignore_zero_occupancy false to change this behavior
INFO:rosetta:core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] PDB reader is ignoring atom  CZ  in residue 141 D.  Pass flag -ignore_zero_occupancy false to change this behavior
INFO:rosetta:core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] PDB reader is ignoring atom  CD  in residue 144 D.  Pass flag -ignore_zero_occupancy false to change this behavior
INFO:rosetta:core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] PDB reader is ignoring atom  CG  in residue 174 D.  Pass flag -ignore_zero_occupancy false to change this behavior
INFO:rosetta:core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] PDB reader is ignoring atom  CD  in residue 212 D.  Pass flag -ignore_zero_occupancy fa

setup complete


Just like before, you can edit the resfile to your own personal specifications. Alternatively, you can use task operations to automate the process. Let's use task operations to fix all residues not at the interface.

## Setting Designable Residues:

Create a new task for design


In [ ]:
from pyrosetta.rosetta.core.select.residue_selector import InterGroupInterfaceByVectorSelector, ChainSelector, NotResidueSelector

chain1 = ChainSelector( "1" ) #selects the first chain
chain2 = ChainSelector( "2" ) #selects the second chain

interface_selector = InterGroupInterfaceByVectorSelector( chain1, chain2 );#selects residues at the interface
not_interface_selector = NotResidueSelector( interface_selector ); #selects residues not at the interface

from pyrosetta.rosetta.core.pack.task.operation import PreventRepackingRLT, RestrictToRepackingRLT, OperateOnResidueSubset

#prevent non interface residues from repacking/designing
fix_non_interface = OperateOnResidueSubset( PreventRepackingRLT(), not_interface_selector )

#perhaps we are performing one-sided design and do not want to make mutations on chain 2:
no_mutation_chain2 = OperateOnResidueSubset( RestrictToRepackingRLT(), chain2 )

from pyrosetta.rosetta.core.pack.task import TaskFactory
task_factory = TaskFactory()
task_factory.push_back( fix_non_interface )
task_factory.push_back( no_mutation_chain2 )

task_design = task_factory.create_task_and_apply_taskoperations( pose )
print( "Num residues: ", pose.size() )
print( "Num packable residues: ", task_design.num_to_be_packed() ) # this includes the ones being designed

num_designable = 0
for i in range( 1, pose.size() + 1 ):
    if( task_design.design_residue( i ) ):
        num_designable += 1;
print( "Num designable residues: ", num_designable )

Num residues:  454
Num packable residues:  116
Num designable residues:  53


## Running HBNet

This is an interface case so we will use HBNetStapleInterface.  We will use both the code-level interface, and the XML interface as an introduction to this functionality.  The XML interface to PyRosetta will be covered more in later workshops.  

In [ ]:
from pyrosetta.rosetta.protocols.hbnet import HBNetStapleInterface

#This is similar to the clone operation, 
# but instead of copying the original pose, we copy the dtat of start pose INTO the pose

pose.assign(start_pose)

# There are two ways to setup a mover:
#   (1) by creating a blank instance of that mover and individually setting any non-default features
#   (2) by passing an XML string, similar to rosetta_scripts
# We will be using option 2 because it provides a more consistent interface to the movers
# Plus, the store_network_scores_in_pose feature is only available using option 2 for versions of PyRosetta older than September-ish 2019
setup_using_string = True #Option 2

if setup_using_string:
    hbnet = pyrosetta.rosetta.protocols.rosetta_scripts.XmlObjects.create_from_string("""
    <MOVERS>
    <HBNetStapleInterface name="hbnet" monte_carlo="true" store_network_scores_in_pose="true"/>
    </MOVERS>
    """).get_mover("hbnet")

    #monte_carlo="true" is highly recommended, especially for large systems like asymmetric interfaces
    #see PMID: 29652499
else:
    hbnet = HBNetStapleInterface()

    #This is highly recommended, especially for large systems like asymmetric interfaces
    #see PMID: 29652499
    hbnet.set_monte_carlo_branch( True )

    #set_monte_carlo_seed_must_be_buried does two things:
    #(1) speeds us up by decreasing the sample space
    #(2) ensures that our final hbond network will be at least partially buried
    #hbnet.set_monte_carlo_seed_must_be_buried( True )
    #commenting this out just to give us more results

#You can toggle the verbosity here if you are interested
#hbnet.verbose( True )

#We can normallly leave this as the default
#making it smaller now to let it run faster
hbnet.set_total_num_mc_runs( 1000 )
#Could have been done with
#<HBNetStapleInterface name="hbnet" monte_carlo="true" total_num_mc_runs="1000" store_network_scores_in_pose="true"/>

hbnet.task_factory( task_factory )
#alternatively:
#hbnet.set_task( task_design )

hbnet.set_score_function( scorefxn )
hbnet.apply(pose)

print( "Change in score", scorefxn(pose) - scorefxn(start_pose) )


INFO:rosetta:protocols.rosetta_scripts.RosettaScriptsParser: Generating XML Schema for rosetta_scripts...
INFO:rosetta:protocols.rosetta_scripts.RosettaScriptsParser: ...done
INFO:rosetta:protocols.rosetta_scripts.RosettaScriptsParser: Initializing schema validator...
INFO:rosetta:protocols.rosetta_scripts.RosettaScriptsParser: ...done
INFO:rosetta:protocols.rosetta_scripts.RosettaScriptsParser: Validating input script...
INFO:rosetta:protocols.rosetta_scripts.RosettaScriptsParser: ...done
INFO:rosetta:protocols.rosetta_scripts.RosettaScriptsParser: Parsed script:
<ROSETTASCRIPTS>
	<MOVERS>
		<HBNetStapleInterface monte_carlo="true" name="hbnet" store_network_scores_in_pose="true"/>
	</MOVERS>
	<PROTOCOLS/>
</ROSETTASCRIPTS>
INFO:rosetta:core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
INFO:rosetta:core.scoring.etable: Starting energy table calculation
INFO:rosetta:core.scoring.etable: smooth_etable: changing atr/rep split to bottom of energy well
INFO:rosetta:core.scoring.eta

Change in score 1209.278530664904


Wait, my score is terrible.

__Question:__ Why?

In [ ]:
# Because it's more precise this way

## Finishing Design:

Well of course the score is terrible, the pose is dense with clashes. We had 116 packable residues and only assigned states to 5 of them. The other 111 residues are still in their input conformations and likely clash with the 5 we just assigned.

We need to run the packer (either using PackRotamersMover or FastDesign) but we don't want to overwrite the residues we just assigned with HBNet. The trick here is to select the residues with "HBNet" labels and fix them.

In [ ]:
from pyrosetta.rosetta.core.select.residue_selector import ResiduePDBInfoHasLabelSelector

#prevent hbnet residues from repacking/designing
hbnet_selector = ResiduePDBInfoHasLabelSelector( "HBNet" )
fix_hbnet = OperateOnResidueSubset( PreventRepackingRLT(), hbnet_selector )
task_factory.push_back( fix_hbnet ) #recycling the same factory as before, just adding a new operation
task_design2 = task_factory.create_task_and_apply_taskoperations( pose )

#sanity check
num_hbnet_residues = 0
for x in hbnet_selector.apply( pose ):
    if x:
        num_hbnet_residues += 1
print( "Num HBNet Residues", num_hbnet_residues )

#this is unrelated to the narrative but I highly recommend using the linear memory interaction graph whenever performing design. It's a huge speedup
#it does not seem to matter for the scope here, but it will when you start using extra chi sampling (-ex1, -ex2)
task_design2.or_linmem_ig( True )

from pyrosetta.rosetta.protocols.minimization_packing import PackRotamersMover
pack_mover = PackRotamersMover( scorefxn, task_design2 )
pack_mover.apply( pose )
print( "Change in score", scorefxn(pose) - scorefxn(start_pose) )

Num HBNet Residues 5


INFO:rosetta:core.pack.pack_rotamers: built 12444 rotamers at 110 positions.
INFO:rosetta:core.pack.interaction_graph.interaction_graph_factory: Instantiating LinearMemoryInteractionGraph


Change in score 158.48651751526


## We made it...?

The change in score is a better, but still positive. One great thing about HBNet is that it can return multiple poses. Each one is slightly worse than the previous by HBNet's standards but might design into something better. Let's try a few to see if they help.

In [ ]:
#there were 10 (or so) networks total, but let's just try the next 5
#this might take a few minutes...
if not os.getenv('DEBUG'): #Adding this line to decrease runtime on the testing server
    for x in range(0,5):
        extra_pose = hbnet.get_additional_output()
        if extra_pose is None:
            break
        task_design3 = task_factory.create_task_and_apply_taskoperations( extra_pose )
        task_design3.or_linmem_ig( True )
        pack_mover = PackRotamersMover( scorefxn, task_design3 )
        pack_mover.apply( extra_pose )
        print( "Change in score", scorefxn(extra_pose) - scorefxn(start_pose) )

INFO:rosetta:protocols.hbnet.HBNet:
INFO:rosetta:core.pack.pack_rotamers: built 12619 rotamers at 111 positions.
INFO:rosetta:core.pack.interaction_graph.interaction_graph_factory: Instantiating LinearMemoryInteractionGraph
INFO:rosetta:protocols.hbnet.HBNet:


Change in score 118.64912865170936


INFO:rosetta:core.pack.pack_rotamers: built 12854 rotamers at 113 positions.
INFO:rosetta:core.pack.interaction_graph.interaction_graph_factory: Instantiating LinearMemoryInteractionGraph
INFO:rosetta:protocols.hbnet.HBNet:


Change in score 24.276594525971575


INFO:rosetta:core.pack.pack_rotamers: built 12879 rotamers at 113 positions.
INFO:rosetta:core.pack.interaction_graph.interaction_graph_factory: Instantiating LinearMemoryInteractionGraph
INFO:rosetta:protocols.hbnet.HBNet:


Change in score 149.1720235493416


INFO:rosetta:core.pack.pack_rotamers: built 11858 rotamers at 111 positions.
INFO:rosetta:core.pack.interaction_graph.interaction_graph_factory: Instantiating LinearMemoryInteractionGraph
INFO:rosetta:protocols.hbnet.HBNet:


Change in score -44.815447481188414


INFO:rosetta:core.pack.pack_rotamers: built 12826 rotamers at 113 positions.
INFO:rosetta:core.pack.interaction_graph.interaction_graph_factory: Instantiating LinearMemoryInteractionGraph


Change in score -51.83529961721587


## But wait, there's more??

Great! We found a few results that designed to me more stable than the input pose (-60, -45, and -31 REU)!

The main score function is not the only way to evaluate these networks. HBNet also adds its own score terms. These are useful for sorting/filtering decoys before running expensive packing calculations.

In [ ]:
from pyrosetta.rosetta.core.pose import hasPoseExtraScore, getPoseExtraScore

if hasPoseExtraScore( pose, "HBNet_NumUnsatHpol" ):
    #All 3 of these metrics are explained in more detail in Maguire, Boyken, et al. (see second reference below)

    #NumUnsatHpol is HBNet's primary sorting metric, it counts the number of polar hydrogen atoms that are unsatisfied (buried and not forming hbonds). We know that there are no heavy (non-hydrogen) unsatisfied atoms because HBNet filters those out by default. Lower is better
    print( "HBNet_NumUnsatHpol", getPoseExtraScore( pose, "HBNet_NumUnsatHpol" ) )

    #HBNet's secondary sorting metric. 1.0 if every polar atom in the network is forming the maximum number of hbonds. Higher is better
    print( "HBNet_Saturation", getPoseExtraScore( pose, "HBNet_Saturation" ) )

    #HBNet's tertiary sorting metric. A little complicated but lower is better.
    print( "HBNet_Score", getPoseExtraScore( pose, "HBNet_Score" ) )
else:
    print( "Somebody go bug a developer to enable this feature for PyRosetta" )

HBNet_NumUnsatHpol 0.0
HBNet_Saturation 0.7142857313156128
HBNet_Score -0.6500769257545471


## Advice for using this in the wild

#### ex1 ex2

HBonds are very sensitive to sidechain sampling resolution. I highly recommend using -ex1 and -ex2. You can do this by adding:
```py
ex1ex2 = ExtraRotamersGeneric()
ex1ex2.ex1( True )
ex1ex2.ex2( True )
task_factory.push_back( ex1ex2 )
```

#### get_additional_output

As we saw in the exercise, the first result out of HBNet does not always wind up being the best. Try designing with a few results from `hbnet.get_additional_output()` to get more coverage of the design space. For the commandline users reading this, this functionality can also be accessed via `multistage_rosetta_scripts` or the `MultiplePoseMover` in `rosetta_scripts`. See the `rosetta_scripts_scripts` repository for examples.

#### set_monte_carlo_seed_must_be_buried

I highly recommend playing with the `set_monte_carlo_seed_must_be_buried` mentioned above. Without it, HBNet tends to just design many surface networks that nobody really cares about.

## Thought Question

The energy of HBNet+Design is often less favorable that the energy after an equivalent design run without HBNet. Why do people still use HBNet?


## References


- Boyken SE, Chen Z, Groves B, et al. De novo design of protein homo-oligomers with modular hydrogen-bond network-mediated specificity. Science. 2016;352(6286):680–687. doi:10.1126/science.aad8865


- Maguire JB, Boyken SE, Baker D, Kuhlman B. Rapid Sampling of Hydrogen Bond Networks for Computational Protein Design. J Chem Theory Comput. 2018;14(5):2751–2760. doi:10.1021/acs.jctc.8b00033

# *De Novo* Parametric Backbone Design

Keywords: Parametric, Bundle, BundleGridSampler, MakeBundle

## Overview

In this workshop, we will give examples for how to do Parametric Protein Design using the RosettaScripts interface to Rosetta written. The code you will be using from Rosetta was written by Vikram Mulligan.  The BundleGridSampler and MakeBundle movers are available through code in PyRosetta, but because of the many number of options to it, it is a bit easier to work with these in XML. We will use this opportunity to do see how easy it is to run XML in PyRosetta, but we will cover the code-level interface as well in this workshop.  

This is some cool stuff. Enjoy. :)

Please refer to the docs for more info.

https://www.rosettacommons.org/docs/latest/scripting_documentation/RosettaScripts/Movers/Movers-RosettaScripts

https://www.rosettacommons.org/docs/latest/scripting_documentation/RosettaScripts/Movers/movers_pages/MakeBundleMover

https://www.rosettacommons.org/docs/latest/scripting_documentation/RosettaScripts/Movers/movers_pages/BundleGridSamplerMover

In [ ]:
import logging
logging.basicConfig(level=logging.INFO)
import pyrosetta

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
pyrosetta.init("-mute all")

INFO:pyrosetta.rosetta:Found rosetta database at: /usr/local/lib/python3.7/dist-packages/pyrosetta/database; using it....
INFO:pyrosetta.rosetta:PyRosetta-4 2021 [Rosetta PyRosetta4.MinSizeRel.python37.ubuntu 2020.50.post.dev+970.commits.3700df145608444753aabbec9c4681ec9b21f74b 2021-02-24T13:24:53] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
INFO:rosetta:core.init: Checking for fconfig files in pwd and ./rosetta/flags
INFO:rosetta:core.init: Rosetta version: PyRosetta4.MinSizeRel.python37.ubuntu r17132 2020.50.post.dev+970.commits.3700df14560 3700df145608444753aabbec9c4681ec9b21f74b http://www.pyrosetta.org 2021-02-24T13:24:53
INFO:rosetta:core.init: command: PyRosetta -mute all -database /usr/local/lib/python3.7/dist-packages/pyrosetta/database
INFO:rosetta:basic.random.init_random_generator: 'RNG device' seed mode, using '/dev/urandom', seed=1786069850 seed_offset=0 real_seed=178606985

PyRosetta-4 2021 [Rosetta PyRosetta4.MinSizeRel.python37.ubuntu 2020.50.post.dev+970.commits.3700df145608444753aabbec9c4681ec9b21f74b 2021-02-24T13:24:53] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.


## Overview

Parametric sampling means changing specific angles of groups of bundles (coiled-coils) in order to sample different conformations.  The equations that are used for Parametric sampling were originally described by Crick and are called Crick equations. 

These equations were reintroduced to the scientific community through William F Degrado's seminal paper, "Probing designability via a generalized model of helical bundle geometry"

Since the puplication of that paper, researchers have used this method to create bundles of all sorts.  Fibers, membrane proteins, and even working antiporters!  


## Main Rosetta References (in order of use of tools)

- Dang B, Wu H, Mulligan VK, Mravic M, Wu Y, Lemmin T, Ford A, Silva DA, Baker D, DeGrado WF.

   **"De novo design of covalently constrained mesosize protein scaffolds with unique tertiary structures."**
   
   *Proc Natl Acad Sci U S A*. 2017 Oct 10;114(41):10852-10857.
   https://www.ncbi.nlm.nih.gov/pubmed/28973862
   
   
- Lu P,Min D, DiMaio F, Wei KY, Vahey MD, Boyken SE, Mulligan, et al.

   **"Accurate computational design of multipass transmembrane proteins."**
   
   *Science*. 2018 Mar 2;359(6379):1042-1046. 
   https://www.ncbi.nlm.nih.gov/pubmed/29496880
   
   
- Chen Z, Boyken SE, Jia M, Busch F, Flores-Solis D, Mulligan, et al.

   **"Programmable design of orthogonal protein heterodimers."**
   
   *Nature*. 2019 Jan;565(7737):106-111. 
   https://www.ncbi.nlm.nih.gov/pubmed/30568301
   
   
- Langan RA Boyken SE, Ng AH, Samson JA, Dods G, Mulligan, et al.

  **"De novo design of bioactive protein switches."**
  
  *Nature*. 2019 Aug;572(7768):205-210.
  https://www.ncbi.nlm.nih.gov/pubmed/31341284
  
  
## Parameter tips (From Vikram himself)

- Most bundles would have r0 from 3 to 10 A (depending on the helix count).
- Omega0 should range from about -2.5 degrees to 2.5 degrees.
- Delta_omega1 can be anywhere from 0 to 360.  (It's just the roll of the helix about its own axis).
- Delta_z0, delta_z1 or delta_t can be anything (you're just sliding the helix up and down).  Typically, you'd just sample around a few Angstroms, though.
- r1, omega1, etc. are all minor helix parameters that are tied to the secondary structure.  We don't normally vary them.

  
## Creating multiple Helices
The Helix lines tell the sampler to create a helix and how to do so. We set them at 90 degree rotations through this line, but we could add additional parameters as well.  These parameters are VERY WELL documented on the link above, so make sure to use that:

**delta_omega0**: Rotation of a helix about the z-axis, stored in radians.

### Changing r0:

Here, we will sample on the r0 axis.  To make this a bit easier, we will set dump_pdbs to 1. You can also put this code in a loop and sample on that as well, writing each to pymol.  You would set r0 instead of r0_max and min.

**r0**: Major helix radius, in Angstroms.

In [ ]:
pose = pyrosetta.rosetta.core.pose.Pose()
pyrosetta.rosetta.protocols.rosetta_scripts.XmlObjects.create_from_string("""
<SCOREFXNS>
  <ScoreFunction name="sfxn1" weights="ref2015"/>
</SCOREFXNS>
<MOVERS>
  <BundleGridSampler name="bgs1"
                     helix_length="20"
                     scorefxn="sfxn1"
                     r0_min="1.0"
                     r0_max="10.0"
                     r0_samples="10"
                     omega0="0.05"
                     delta_omega0="0"
                     delta_omega1="0"
                     delta_t="0"
                     dump_pdbs="1">
  <Helix/>
  <Helix delta_omega0="3.14" r0_copies_helix="1"/>
</BundleGridSampler>
</MOVERS>
""").get_mover("bgs1").apply(pose)

INFO:rosetta:protocols.rosetta_scripts.RosettaScriptsParser: Generating XML Schema for rosetta_scripts...
INFO:rosetta:protocols.rosetta_scripts.RosettaScriptsParser: ...done
INFO:rosetta:protocols.rosetta_scripts.RosettaScriptsParser: Initializing schema validator...
INFO:rosetta:protocols.rosetta_scripts.RosettaScriptsParser: ...done
INFO:rosetta:protocols.rosetta_scripts.RosettaScriptsParser: Validating input script...
INFO:rosetta:protocols.rosetta_scripts.RosettaScriptsParser: ...done
INFO:rosetta:protocols.rosetta_scripts.RosettaScriptsParser: Parsed script:
<ROSETTASCRIPTS>
	<SCOREFXNS>
		<ScoreFunction name="sfxn1" weights="ref2015"/>
	</SCOREFXNS>
	<MOVERS>
		<BundleGridSampler delta_omega0="0" delta_omega1="0" delta_t="0" dump_pdbs="1" helix_length="20" name="bgs1" omega0="0.05" r0_max="10.0" r0_min="1.0" r0_samples="10" scorefxn="sfxn1">
			<Helix/>
			<Helix delta_omega0="3.14" r0_copies_helix="1"/>
		</BundleGridSampler>
	</MOVERS>
	<PROTOCOLS/>
</ROSETTASCRIPTS>
INFO:rose

In [ ]:
view_pose(pose)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Ok, so now lets have a look at this - it is quite fast as we are simply manipulating the backbone and doing doing any sequence design (yet).  Which axis changed?  Lets try changing some other values!
Note that if we have dump_pdbs on - we will be overwriting them, so make sure to move the current ones to another directory if you 

## Changing omega0

This time, we will change the parameters a bit through code and observe them by directly dumping them or through the PyMolMover.
Instead of the BundleGridSampler, we will use the MakeBundle mover.

Note the use of Python3's awesome f-strings here, and the speed.  Because we need to parse the xml object and create a new mover, pose, etc. each time, this is much slower than the sampler - but it does allow us a bit more flexibility to use python the way it was intended.  Once again, the `MakeBundle` mover does not have an adaquite code-level interface to allow us to set variables the traditional way. If you end up writing C++ Rosetta code, make sure to always keep the XML and code-level interfaces feature-complete!

**omega0**: Major helix twist per residue, stored in radians.

In [ ]:
import math


for i in range(0, 32, 2):

    i_rad = math.radians(i)
    pose = pyrosetta.rosetta.core.pose.Pose()

    sampler_string = f'''
    <SCOREFXNS>
        <ScoreFunction name="sfxn1" weights="ref2015"/>
    </SCOREFXNS>
    <MOVERS>
        <MakeBundle name="bgs1"
                     helix_length="20"
                     r0="5"
                     omega0="{i_rad}"
                     delta_omega0="0"
                     delta_omega1="0"
                     delta_t="0">
        <Helix/>
        <Helix delta_omega0="3.14" r0_copies_helix="1"/>
        
        </MakeBundle>
    </MOVERS>'''
    pyrosetta.rosetta.protocols.rosetta_scripts.XmlObjects.create_from_string(sampler_string).get_mover("bgs1").apply(pose)
    pose.dump_pdb('outputs/omega0_'+str(i)+'.pdb')

INFO:rosetta:protocols.rosetta_scripts.RosettaScriptsParser: Generating XML Schema for rosetta_scripts...
INFO:rosetta:protocols.rosetta_scripts.RosettaScriptsParser: ...done
INFO:rosetta:protocols.rosetta_scripts.RosettaScriptsParser: Initializing schema validator...
INFO:rosetta:protocols.rosetta_scripts.RosettaScriptsParser: ...done
INFO:rosetta:protocols.rosetta_scripts.RosettaScriptsParser: Validating input script...
INFO:rosetta:protocols.rosetta_scripts.RosettaScriptsParser: ...done
INFO:rosetta:protocols.rosetta_scripts.RosettaScriptsParser: Parsed script:
<ROSETTASCRIPTS>
	<SCOREFXNS>
		<ScoreFunction name="sfxn1" weights="ref2015"/>
	</SCOREFXNS>
	<MOVERS>
		<MakeBundle delta_omega0="0" delta_omega1="0" delta_t="0" helix_length="20" name="bgs1" omega0="0.0" r0="5">
			<Helix/>
			<Helix delta_omega0="3.14" r0_copies_helix="1"/>
		</MakeBundle>
	</MOVERS>
	<PROTOCOLS/>
</ROSETTASCRIPTS>
INFO:rosetta:core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
INFO:rosetta:basic.i

What is the maximum value of omega0?  Which PDBs are empty??

In [ ]:
## iterate through all PDBs you've created and send them to viewer
import pyrosetta.distributed.viewer as viewer
omega_pose = pyrosetta.pose_from_pdb('outputs/omega0_0.pdb')
omega_pose_2 = pyrosetta.pose_from_pdb('outputs/omega0_2.pdb')
view = viewer.init([omega_pose, omega_pose_1]) + viewer.setStyle()
view()

INFO:rosetta:core.import_pose.import_pose: File 'outputs/omega0_0.pdb' automatically determined to be of type PDB
INFO:rosetta:core.conformation.Conformation: [ WARNING ] missing heavyatom:  OXT on residue ALA:CtermProteinFull 20
INFO:rosetta:core.conformation.Conformation: [ WARNING ] missing heavyatom:  OXT on residue ALA:CtermProteinFull 40
INFO:rosetta:core.import_pose.import_pose: File 'outputs/omega0_2.pdb' automatically determined to be of type PDB
INFO:rosetta:core.conformation.Conformation: [ WARNING ] missing heavyatom:  OXT on residue ALA:CtermProteinFull 20
INFO:rosetta:core.conformation.Conformation: [ WARNING ] missing heavyatom:  OXT on residue ALA:CtermProteinFull 40


interactive(children=(IntSlider(value=0, continuous_update=False, description='Decoys', max=1), Output()), _do…

<function pyrosetta.distributed.viewer.core.Viewer.show.<locals>.view>

## MakeBundle Mover in PyRosetta

Ok, we have an idea for how to use code to create some helical backbones.  Lets try this through code instead of RosettaScripts.

Note that the BPC_xx are `ENUMS` as we have seen before and are imported when we import all of the helical bundle namespace.  
Also, if you call `mb.set_use_degrees(True)` before creating helices, you won't need the `math.radians()` call.

In [ ]:
from pyrosetta.rosetta.protocols.helical_bundle import *
import math

pose = pyrosetta.rosetta.core.pose.Pose()
mb = MakeBundle()

mb.add_helix()
mb.add_helix()
mb.helix(2).calculator_op().real_parameter(BPC_delta_omega0).set_value(3.14)

for i in range(1, 3):
    mb.helix(i).set_helix_length(20)
    mb.helix(i).calculator_op().real_parameter(BPC_r0).set_value(5)
    mb.helix(i).calculator_op().real_parameter(BPC_omega0).set_value(math.radians(12))

mb.apply(pose)
pose.dump_pdb("outputs/manual_make_bundle.pdb")
    
    

INFO:rosetta:basic.io.database: Database file opened: protocol_data/crick_parameters/alpha_helix.crick_params
INFO:rosetta:basic.io.database: Database file opened: protocol_data/crick_parameters/alpha_helix.crick_params


True

In [ ]:
view = viewer.init(pose) + viewer.setStyle()
view()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## Conclusion

This should give you an overview of how to use the MakeBundle and BundleGridSampler within PyRosetta.  Everything else is simply parameters.  Read up and you'll be ready to start designing your own helical bundles. 

## References:

- **Bundle Tools**: Vikram Mulligan
- Grigoryan, Gevorg and William F Degrado. “Probing designability via a generalized model of helical bundle geometry” Journal of molecular biology vol. 405,4 (2010): 1079-100. https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3052747/
- Huang, Po-Ssu et al. “High thermodynamic stability of parametrically designed helical bundles” Science (New York, N.Y.) vol. 346,6208 (2014): 481-485. https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4612401/
- Weitzner, Brian. "Parametric disasters" Published September 03, 2018. https://weitzner.github.io/posts/2018/08/parametric-disasters/

# *De Novo* Protein Design with PyRosetta
Keywords:

## Overview

Before we begin, we must install the biopython package; it doesn't seem to require a "restart notebook" in order to be imported successfully in the cell after.

In [ ]:
!pip install biopython --user

     |████████████████████████████████| 2.3MB 6.7MB/s 


In [ ]:
import Bio.Data.IUPACData as IUPACData
import Bio.SeqUtils
import logging
logging.basicConfig(level=logging.INFO)
import os
import pyrosetta
import pyrosetta.distributed
import pyrosetta.distributed.viewer as viewer
import site
import sys

Initialize PyRosetta:

In [ ]:
flags = """
-linmem_ig 10
-ignore_unrecognized_res 1
-mute core.select.residue_selector.SecondaryStructureSelector
-mute core.select.residue_selector.PrimarySequenceNeighborhoodSelector
-mute protocols.DsspMover
"""
pyrosetta.distributed.init(flags)

 Let's setup the pose of a *de novo* helical bundle from the PDB for downstream design: https://www.rcsb.org/structure/5J0J

In [ ]:
start_pose = pyrosetta.io.pose_from_file("inputs/5J0J.clean.pdb")
pose = start_pose.clone()

INFO:rosetta:core.import_pose.import_pose: File 'inputs/5J0J.clean.pdb' automatically determined to be of type PDB
INFO:rosetta:core.conformation.Conformation: [ WARNING ] missing heavyatom:  OXT on residue ALA:CtermProteinFull 70
INFO:rosetta:core.conformation.Conformation: [ WARNING ] missing heavyatom:  OXT on residue ALA:CtermProteinFull 139
INFO:rosetta:core.conformation.Conformation: [ WARNING ] missing heavyatom:  OXT on residue ALA:CtermProteinFull 210


## Design Strategy 

Minimize the crystal structure coordinates, then convert chain "A" to poly-alanine, and then perform one-sided protein-protein interface design designing chain A while only re-packing the homotrimer interface residues of chains B and C! Therefore, we are designing a homotrimer into a heterotrimer. Furthermore, prevent backbone torsions from minimizing and only minimize the side-chains of the homotrimer interface and all of chain A using the `FastDesign` mover! After design, repack and minimize all side-chains.

 Prior to and after design, we want to relax the protein with a scorefunction that packs the rotamers and minimizes the side-chain degrees of freedom while optimizing for realistic energies. If you were to allow backbone minimization (which you may optionally choose to), we would want use a Cartesian scorefunction (in this case, `ref2015_cart.wts`) which automatically sets `cart_bonded` scoreterm to a weight of 1.0, which helps to close chain breaks in the backbone. Similarly, you might want to also turn on the `coordinate_constraint` scoreterm to penalize deviations of the backbone coordinates from their initial coordinates during minimization. In this tutorial, we demonstrate that concept but will prevent backbone torsions from being minimized (however, feel free to turn on backbone minimization!):

In [ ]:
relax_scorefxn = pyrosetta.create_score_function("ref2015_cart.wts")
relax_scorefxn.set_weight(pyrosetta.rosetta.core.scoring.ScoreType.coordinate_constraint, 1.0)
print("The starting pose total_score is {}".format(relax_scorefxn(start_pose)))

INFO:rosetta:core.energy_methods.CartesianBondedEnergy: Creating new peptide-bonded energy container (210)


The starting pose total_score is 2113.581004929075


 For *design*, if we allowed backbone minimzation we again would turn on the `coordinate_constraint` scoreterm to penalize deviations of the backbone coordinates from their initial coordinates during minimization. Additionally, we will use "non-pairwise decomposable" scoreterms to guide the packer trajectories (i.e. fixed backbone sequence design trajectories) in favor of our hypothetical design requirements to solve our biological problem. In this tutorial, we will make use of the following additional scoreterms during design:
- `aa_composition`: This scoring term is intended for use during design, to penalize deviations from a desired residue type composition. Applies to any amino acid composition requirements specified by the user within a ResidueSelector. This scoreterm also applies to the `AddHelixSequenceConstraints` mover which sets up ideal sequence constraints for each helix in a pose or in a selection.
- `voids_penalty`: This scoring term is intended for use during design, to penalize buried voids or cavities and to guide the packer to design solutions in which all buried volume is filled with side-chains.
- `aa_repeat`: This wholebody scoring term is intended for use during design, to penalize long stretches in which the same residue type repeats over and over (e.g. poly-Q sequences). 
- `buried_unsatisfied_penalty`: This scoring term is intended for use during design, to provide a penalty for buried hydrogen bond donors or acceptors that are unsatisfied. 
- `netcharge`: This scoring term is intended for use during design, to penalize deviations from a desired net charge in a pose or in a selection.
- `hbnet`: This scoring term is intended for use during design, to provide a bonus for hydrogen bond network formation.

In [ ]:
design_scorefxn = pyrosetta.create_score_function("ref2015_cart.wts")
design_scorefxn.set_weight(pyrosetta.rosetta.core.scoring.ScoreType.coordinate_constraint, 10.0)
design_scorefxn.set_weight(pyrosetta.rosetta.core.scoring.ScoreType.aa_composition, 1.0)
design_scorefxn.set_weight(pyrosetta.rosetta.core.scoring.ScoreType.voids_penalty, 0.25)
design_scorefxn.set_weight(pyrosetta.rosetta.core.scoring.ScoreType.aa_repeat, 1.0)
design_scorefxn.set_weight(pyrosetta.rosetta.core.scoring.ScoreType.buried_unsatisfied_penalty, 1.0)
design_scorefxn.set_weight(pyrosetta.rosetta.core.scoring.ScoreType.hbnet, 1.0)
design_scorefxn.set_weight(pyrosetta.rosetta.core.scoring.ScoreType.netcharge, 1.0)
print("The starting pose total_score is {}".format(design_scorefxn(start_pose)))

INFO:rosetta:basic.io.database: Database file opened: scoring/score_functions/aa_repeat_energy/default_repeat_penalty_table.rpt_pen


The starting pose total_score is 23016.581004929074


 By using the `relax_scorefxn` before and after the `design_scorefxn`, we ensure that these "non-pairwise decomposable" scoreterms are not forcing unrealistic rotamers that would otherwise not be held in place without these additional scoreterms.

 Prior to any deviation from crystal structure coordinates, apply coordinate constraints to all of the backbone heavy atoms:

In [ ]:
true_selector = pyrosetta.rosetta.core.select.residue_selector.TrueResidueSelector() # Select all residues

# Apply a virtual root onto the pose to prevent large lever-arm effects while minimizing with coordinate constraints
virtual_root = pyrosetta.rosetta.protocols.simple_moves.VirtualRootMover()
virtual_root.set_removable(True)
virtual_root.set_remove(False)
virtual_root.apply(pose)

# Construct the CoordinateConstraintGenerator
coord_constraint_gen = pyrosetta.rosetta.protocols.constraint_generator.CoordinateConstraintGenerator()
coord_constraint_gen.set_id("contrain_all_backbone_atoms!")
coord_constraint_gen.set_ambiguous_hnq(False)
coord_constraint_gen.set_bounded(False)
coord_constraint_gen.set_sidechain(False)
coord_constraint_gen.set_sd(1.0) # Sets a standard deviation of contrained atoms to (an arbitrary) 1.0 Angstroms RMSD. Set higher or lower for different results.
coord_constraint_gen.set_ca_only(False)
coord_constraint_gen.set_residue_selector(true_selector)

# Apply the CoordinateConstraintGenerator using the AddConstraints mover
add_constraints = pyrosetta.rosetta.protocols.constraint_generator.AddConstraints()
add_constraints.add_generator(coord_constraint_gen)
add_constraints.apply(pose)

 Prior to design, minimize with the `FastRelax` mover to optimize the pose within the `relax_scorefxn` scorefunction. Note: this takes ~1min 10s

In [ ]:
tf = pyrosetta.rosetta.core.pack.task.TaskFactory()
tf.push_back(pyrosetta.rosetta.core.pack.task.operation.InitializeFromCommandline())
tf.push_back(pyrosetta.rosetta.core.pack.task.operation.IncludeCurrent())
tf.push_back(pyrosetta.rosetta.core.pack.task.operation.NoRepackDisulfides())
tf.push_back(pyrosetta.rosetta.core.pack.task.operation.OperateOnResidueSubset(
    pyrosetta.rosetta.core.pack.task.operation.PreventRepackingRLT(), true_selector)) # Set to RestrictToRepackingRLT for slower/better results
mm = pyrosetta.rosetta.core.kinematics.MoveMap()
mm.set_bb(False) # Set to true if desired
mm.set_chi(True)
mm.set_jump(False)
fast_relax = pyrosetta.rosetta.protocols.relax.FastRelax(scorefxn_in=relax_scorefxn, standard_repeats=1) # 2-5 repeats suggested for real applications
fast_relax.cartesian(True)
fast_relax.set_task_factory(tf)
fast_relax.set_movemap(mm)
fast_relax.minimize_bond_angles(True)
fast_relax.minimize_bond_lengths(True)
fast_relax.min_type("lbfgs_armijo_nonmonotone")
fast_relax.ramp_down_constraints(False)

if not os.getenv("DEBUG"):
    %time fast_relax.apply(pose)

# Optionally, instead of running this you could reload the saved pose from a previously run trajectory:
#pose = pyrosetta.pose_from_file("minimized_start_pose.pdb")

INFO:rosetta:core.energy_methods.CartesianBondedEnergy: Creating new peptide-bonded energy container (211)
INFO:rosetta:core.energy_methods.CartesianBondedEnergy: Adding undefined angle VRT: X,ORIG,Y to DB with theta0 = 1.5708 , Ktheta = 80
INFO:rosetta:core.energy_methods.CartesianBondedEnergy: Adding undefined length VRT: ORIG,X, to DB with d0 = 1 , Kd = 300
INFO:rosetta:core.energy_methods.CartesianBondedEnergy: Adding undefined length VRT: ORIG,Y, to DB with d0 = 1 , Kd = 300
INFO:rosetta:protocols.relax.FastRelax: CMD: repeat  2113.58  0  0  0.55
INFO:rosetta:protocols.relax.FastRelax: CMD: coord_cst_weight  2113.58  0  0  0.55
INFO:rosetta:protocols.relax.FastRelax: CMD: scale:fa_rep  2001.9  0  0  0.022
INFO:rosetta:core.pack.task: Packer task: initialize from command line()
INFO:rosetta:core.pack.pack_rotamers: built 0 rotamers at 0 positions.
INFO:rosetta:core.pack.interaction_graph.interaction_graph_factory: Instantiating DensePDInteractionGraph
INFO:rosetta:protocols.relax.F

CPU times: user 1min 27s, sys: 615 ms, total: 1min 28s
Wall time: 1min 27s


 Let's check the delta `total_score` per residue after minimizing in the `relax_scorefxn` scorefunction:

In [ ]:
if not os.getenv("DEBUG"):
    initial_score_res = relax_scorefxn(start_pose)/start_pose.size()
    final_score_res = relax_scorefxn(pose)/pose.size()
    delta_total_score_res = final_score_res - initial_score_res
    print("{0} kcal/(mol*res) - {1} kcal/(mol*res) = {2} kcal/(mol*res)".format(final_score_res, initial_score_res, delta_total_score_res))

2.4553280277730765 kcal/(mol*res) - 10.064671452043214 kcal/(mol*res) = -7.609343424270138 kcal/(mol*res)


 We can see that the crystal structure coordinates were not quite optimal according to the `relax_scorefxn` scorefunction. So which model is correct?

By how many Angstroms RMSD did the backbone Cα atoms move?

In [ ]:
pyrosetta.rosetta.core.scoring.CA_rmsd(start_pose, pose)

0.0

 For downstream analysis, we want to save the pose:

In [ ]:
minimized_start_pose = pose.clone()
pyrosetta.dump_pdb(minimized_start_pose, "outputs/minimized_start_pose.pdb")

True

## *De Novo* Protein Design

Prior to designing chain A, first let's make chain A poly-alanine so that we can re-design the sidechains onto the backbone:

In [ ]:
# Since we will do direct pose manipulation, first remove the constraints
remove_constraints = pyrosetta.rosetta.protocols.constraint_generator.RemoveConstraints()
remove_constraints.add_generator(coord_constraint_gen)
remove_constraints.apply(pose)

 Obtain chain A and convert it to poly-alanine

In [ ]:
keep_chA = pyrosetta.rosetta.protocols.grafting.simple_movers.KeepRegionMover(res_start=str(pose.chain_begin(1)), res_end=str(pose.chain_end(1)))
keep_chA.apply(pose)
polyA_chA = pyrosetta.rosetta.protocols.pose_creation.MakePolyXMover(aa="ALA", keep_pro=0, keep_gly=0, keep_disulfide_cys=0)
polyA_chA.apply(pose)

 Obtain chains B and C

In [ ]:
pose_chBC = minimized_start_pose.clone()
keep_chBC = pyrosetta.rosetta.protocols.grafting.simple_movers.KeepRegionMover(res_start=str(pose_chBC.chain_begin(2)), res_end=str(pose_chBC.chain_end(3)-1))
keep_chBC.apply(pose_chBC)

 Append chains B and C onto the poly-alanine version of chain A

In [ ]:
pyrosetta.rosetta.core.pose.append_pose_to_pose(pose1=pose, pose2=pose_chBC, new_chain=True)

 Pose is now considered to have only 2 chains.

In [ ]:
pose.num_chains()

2

 Let's re-establish that there are 3 chains.

In [ ]:
switch_chains = pyrosetta.rosetta.protocols.simple_moves.SwitchChainOrderMover()
switch_chains.chain_order("12")
switch_chains.apply(pose)

print(pose.pdb_info())
print("Now the number of chains = {}".format(pose.num_chains()))

INFO:rosetta:core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015


PDB file name: 
 Pose Range  Chain    PDB Range  |   #Residues         #Atoms

0001 -- 0070    A 0001  -- 0070  |   0070 residues;    00703 atoms
0071 -- 0139    B 0071  -- 0139  |   0069 residues;    01187 atoms
0140 -- 0209    C 0140  -- 0209  |   0070 residues;    01212 atoms
                           TOTAL |   0209 residues;    03102 atoms

Now the number of chains = 3


 Re-apply backbone coordinate constraints:

In [ ]:
virtual_root.apply(pose)
add_constraints.apply(pose)

 Have a look at the new pose, chain A in which is ready to be designed!

In [ ]:
view_pose(pose)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

 Next, we need to apply certain movers with our design specifications to activate certain non-pairwise decomposable scoreterms in the `design_scorefxn` scorefunction, that will be implemented when we run the `FastDesign` mover (or any downstream mover that calls the packer).

 We will frequently be using the following residue selectors:

In [ ]:
chain_A = pyrosetta.rosetta.core.select.residue_selector.ChainSelector("A")
chain_BC = pyrosetta.rosetta.core.select.residue_selector.NotResidueSelector(chain_A)

 Apply the `AddCompositionConstraintMover` mover, which utilizes the `aa_composition` scoreterm. Applying the following mover to the pose imposes the design constraints that the ResidueSelector have 40% aliphatic or aromatic residues other than leucine (i.e. ALA, PHE, ILE, MET, PRO, VAL, TRP, or TYR), and 5% leucines. For documentation, see: https://www.rosettacommons.org/docs/latest/rosetta_basics/scoring/AACompositionEnergy

In [ ]:
add_composition_constraint = pyrosetta.rosetta.protocols.aa_composition.AddCompositionConstraintMover()
add_composition_constraint.create_constraint_from_file_contents("""
PENALTY_DEFINITION
OR_PROPERTIES AROMATIC ALIPHATIC
NOT_TYPE LEU
FRACT_DELTA_START -0.05
FRACT_DELTA_END 0.05
PENALTIES 1 0 1 # The above two lines mean that if we're 5% below or 5% above the desired content, we get a 1-point penalty.
FRACTION 0.4 # Forty percent aromatic or aliphatic, but not leucine
BEFORE_FUNCTION CONSTANT
AFTER_FUNCTION CONSTANT
END_PENALTY_DEFINITION

PENALTY_DEFINITION
TYPE LEU
DELTA_START -1
DELTA_END 1
PENALTIES 1 0 1
FRACTION 0.05 # Five percent leucine
BEFORE_FUNCTION CONSTANT
AFTER_FUNCTION CONSTANT
END_PENALTY_DEFINITION
""")
add_composition_constraint.add_residue_selector(chain_A)
add_composition_constraint.apply(pose)

 Apply the `AddHelixSequenceConstraints` mover, which utilizes the `aa_composition` scoreterm. By default, this mover adds five types of sequence constraints to the designable residues in each alpha helix in the pose. Any of these behaviours may be disabled or modified by invoking advanced options, but no advanced options need be set in most cases. The five types of sequence constraints are:

- A strong sequence constraint requiring at least two negatively-charged residues in the first (N-terminal) three residues of each alpha-helix.

- A strong sequence constraint requiring at least two positively-charged residues in the last (C-terminal) three residues of each alpha-helix.

- A weak but strongly ramping sequence constraint penalizing helix-disfavoring residue types (by default, Asn, Asp, Ser, Gly, Thr, and Val) throughout each helix. (A single such residue is sometimes tolerated, but the penalty for having more than one residue in this category increases quadratically with the count of helix-disfavouring residues.)

- A weak sequence constraint coaxing the helix to have 10% alanine. Because this constraint is weak, deviations from this value are tolerated, but this should prevent an excessive abundance of alanine residues.

- A weak sequence constraint coaxing the helix to have at least 25% hydrophobic content. This constraint is also weak, so slightly less hydrophobic helices will be tolerated to some degree. Note that alanine is not considered to be "hydrophobic" within Rosetta. 

### For documentation, see: https://www.rosettacommons.org/docs/latest/rosetta_basics/scoring/AACompositionEnergy

In [ ]:
add_helix_sequence_constraints = pyrosetta.rosetta.protocols.aa_composition.AddHelixSequenceConstraintsMover()
add_helix_sequence_constraints.set_residue_selector(chain_A)
add_helix_sequence_constraints.apply(pose)

Note: the `aa_repeat` scoreterm works out-of-the-box, and does not need to be applied to the pose to work, it just needs to have a weight of >0 in the scorefunction used by the packer. It imposes a penalty for each stretch of repeating amino acids, with the penalty value depending nonlinearly on the length of the repeating stretch. By default, 1- or 2-residue stretches incur no penalty, 3-residue stretches incur a penalty of +1, 4-residue stretches incur a penalty of +10, and 5-residue stretches or longer incur a penalty of +100. Since the term is sequence-based, it is really only useful for design -- that is, it will impose an identical penalty for a fixed-sequence pose, regardless its conformation. This also means that the term has no conformational derivatives: the minimizer ignores it completely. The term is not pairwise-decomposible, but has been made packer-compatible, so it can direct the sequence composition during a packer run. For documentation, see: https://www.rosettacommons.org/docs/latest/rosetta_basics/scoring/Repeat-stretch-energy

 Similarly, the `voids_penalty` scoreterm does not need to be applied to the pose to work, it just needs to have a weight of >0 in the scorefunction used by the packer. For documentation, see: https://www.rosettacommons.org/docs/latest/rosetta_basics/scoring/VoidsPenaltyEnergy

 Similarly, the `buried_unsatisfied_penalty` scoreterm does not need to be applied to the pose to work, it just needs to have a weight of >0 in the scorefunction used by the packer. For documentation, see: https://www.rosettacommons.org/docs/latest/rosetta_basics/scoring/BuriedUnsatPenalty

 Similarly, the `hbnet` scoreterm does not need to be applied to the pose to work, it just needs to have a weight of >0 (ideally between 1.0 to 10.0) in the scorefunction used by the packer. For documentation, see: https://www.rosettacommons.org/docs/latest/rosetta_basics/scoring/HBNetEnergy

 Apply the `AddNetChargeConstraintMover` mover, which utilizes the `netcharge` scoreterm. In this case, we require that the net charge in chain A must be exactly 0.

In [ ]:
add_net_charge_constraint = pyrosetta.rosetta.protocols.aa_composition.AddNetChargeConstraintMover()
add_net_charge_constraint.create_constraint_from_file_contents("""
DESIRED_CHARGE 0 #Desired net charge is zero.
PENALTIES_CHARGE_RANGE -1 1 #Penalties are listed in the observed net charge range of -1 to +1.
PENALTIES 1 0 1 #The penalties are 1 for an observed charge of -1, 0 for an observed charge of 0, and 1 for an observed charge of +1.
BEFORE_FUNCTION QUADRATIC #Ramp quadratically for observed net charges of -2 or less.
AFTER_FUNCTION QUADRATIC #Ramp quadratically for observed net charges of +2 or greater.
""")
add_net_charge_constraint.add_residue_selector(chain_A)
add_net_charge_constraint.apply(pose)

 Specify a custom `relaxscript` that optimizes the `ramp_repack_min` weights to prevent too many alanines from being designed (demonstrated at Pre-RosettaCON 2018):

 Specify TaskOperations to be applied to chain A. In this case, let's use the latest LayerDesign implementation (Note: this is still being actively developed) using the XmlObjects class.

In [ ]:
layer_design_task = pyrosetta.rosetta.protocols.rosetta_scripts.XmlObjects.create_from_string("""
<RESIDUE_SELECTORS>
  <Layer name="surface" select_core="false" select_boundary="false" select_surface="true" use_sidechain_neighbors="true"/>
  <Layer name="boundary" select_core="false" select_boundary="true" select_surface="false" use_sidechain_neighbors="true"/>
  <Layer name="core" select_core="true" select_boundary="false" select_surface="false" use_sidechain_neighbors="true"/>
  <SecondaryStructure name="sheet" overlap="0" minH="3" minE="2" include_terminal_loops="false" use_dssp="true" ss="E"/>
  <SecondaryStructure name="entire_loop" overlap="0" minH="3" minE="2" include_terminal_loops="true" use_dssp="true" ss="L"/>
  <SecondaryStructure name="entire_helix" overlap="0" minH="3" minE="2" include_terminal_loops="false" use_dssp="true" ss="H"/>
  <And name="helix_cap" selectors="entire_loop">
   <PrimarySequenceNeighborhood lower="1" upper="0" selector="entire_helix"/>
  </And>
  <And name="helix_start" selectors="entire_helix">
    <PrimarySequenceNeighborhood lower="0" upper="1" selector="helix_cap"/>
  </And>
  <And name="helix" selectors="entire_helix">
    <Not selector="helix_start"/>
  </And>
  <And name="loop" selectors="entire_loop">
    <Not selector="helix_cap"/>
  </And>
</RESIDUE_SELECTORS>
<TASKOPERATIONS>
  <DesignRestrictions name="layer_design">
    <Action selector_logic="surface AND helix_start" aas="EHKPQR"/>
    <Action selector_logic="surface AND helix" aas="EHKQR"/>
    <Action selector_logic="surface AND sheet" aas="DEHKNQRST"/>
    <Action selector_logic="surface AND loop" aas="DEGHKNPQRST"/>
    <Action selector_logic="boundary AND helix_start" aas="ADEIKLMNPQRSTVWY"/>
    <Action selector_logic="boundary AND helix" aas="ADEIKLMNQRSTVWY"/>
    <Action selector_logic="boundary AND sheet" aas="DEFIKLNQRSTVWY"/>
    <Action selector_logic="boundary AND loop" aas="ADEFGIKLMNPQRSTVWY"/>
    <Action selector_logic="core AND helix_start" aas="AFILMPVWY"/>
    <Action selector_logic="core AND helix" aas="AFILMVWY"/>
    <Action selector_logic="core AND sheet" aas="FILVWY"/>
    <Action selector_logic="core AND loop" aas="AFGILMPVWY"/>
    <Action selector_logic="helix_cap" aas="DNST"/>
  </DesignRestrictions>
</TASKOPERATIONS>
""").get_task_operation("layer_design")

INFO:rosetta:protocols.rosetta_scripts.RosettaScriptsParser: Generating XML Schema for rosetta_scripts...
INFO:rosetta:protocols.rosetta_scripts.RosettaScriptsParser: ...done
INFO:rosetta:protocols.rosetta_scripts.RosettaScriptsParser: Initializing schema validator...
INFO:rosetta:protocols.rosetta_scripts.RosettaScriptsParser: ...done
INFO:rosetta:protocols.rosetta_scripts.RosettaScriptsParser: Validating input script...
INFO:rosetta:protocols.rosetta_scripts.RosettaScriptsParser: ...done
INFO:rosetta:protocols.rosetta_scripts.RosettaScriptsParser: Parsed script:
<ROSETTASCRIPTS>
	<RESIDUE_SELECTORS>
		<Layer name="surface" select_boundary="false" select_core="false" select_surface="true" use_sidechain_neighbors="true"/>
		<Layer name="boundary" select_boundary="true" select_core="false" select_surface="false" use_sidechain_neighbors="true"/>
		<Layer name="core" select_boundary="false" select_core="true" select_surface="false" use_sidechain_neighbors="true"/>
		<SecondaryStructure in

 Also prepare a ResidueSelector for the heterotrimer interface within chains B and C, and the rest of chains B and C. We will use these with `RestrictAbsentCanonicalAASRLT`, `RestrictToRepackingRLT`, and `PreventRepackingRLT` TaskOperations:

In [ ]:
interface = pyrosetta.rosetta.core.select.residue_selector.InterGroupInterfaceByVectorSelector()
interface.group1_selector(chain_A)
interface.group2_selector(chain_BC)
chain_BC_interface = pyrosetta.rosetta.core.select.residue_selector.AndResidueSelector(interface, chain_BC)
not_chain_BC_interface = pyrosetta.rosetta.core.select.residue_selector.NotResidueSelector(chain_BC_interface)
chain_BC_not_interface = pyrosetta.rosetta.core.select.residue_selector.AndResidueSelector(not_chain_BC_interface, chain_BC)

 For minimization, we also need the following ResidueSelector:

In [ ]:
chain_A_and_BC_interface = pyrosetta.rosetta.core.select.residue_selector.OrResidueSelector(chain_A, chain_BC_interface)

 Make sure each ResidueSelector selects the regions as desired (in the following case, the `ResidueSelector` `interface` is visualized:

In [ ]:
view = viewer.init(pose) \
    + viewer.setStyle() \
    + viewer.setStyle(residue_selector=interface, colorscheme="greyCarbon")
view()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

 Create TaskFactory to be used with the `FastRelax` mover (We use this instead of the FastDesign mover as the constructor allows us to set a relax script):

In [ ]:
tf.clear()
tf = pyrosetta.rosetta.core.pack.task.TaskFactory()

tf.push_back(pyrosetta.rosetta.core.pack.task.operation.InitializeFromCommandline())
tf.push_back(pyrosetta.rosetta.core.pack.task.operation.IncludeCurrent())
tf.push_back(pyrosetta.rosetta.core.pack.task.operation.NoRepackDisulfides())

# Prevent repacking on chain_BC_not_interface
tf.push_back(pyrosetta.rosetta.core.pack.task.operation.OperateOnResidueSubset(
    pyrosetta.rosetta.core.pack.task.operation.PreventRepackingRLT(), chain_BC_not_interface))

# Repack only on chain_BC_interface
tf.push_back(pyrosetta.rosetta.core.pack.task.operation.OperateOnResidueSubset(
    pyrosetta.rosetta.core.pack.task.operation.RestrictToRepackingRLT(), chain_BC_interface))

# Enable design on chain_A
aa_to_design = pyrosetta.rosetta.core.pack.task.operation.RestrictAbsentCanonicalAASRLT()
aa_to_design.aas_to_keep("ACDEFGHIKLMNPQRSTVWY")
tf.push_back(pyrosetta.rosetta.core.pack.task.operation.OperateOnResidueSubset(
    aa_to_design, chain_A))

# Apply layer design
tf.push_back(layer_design_task)

# Convert the task factory into a PackerTask
packer_task = tf.create_task_and_apply_taskoperations(pose)
# View the PackerTask
print(packer_task)

INFO:rosetta:core.pack.task: Packer task: initialize from command line()


#Packer_Task

Threads to request: ALL AVAILABLE

resid	pack?	design?	allowed_aas
1	TRUE	TRUE	ASP:NtermProteinFull,ASN:NtermProteinFull,SER:NtermProteinFull,THR:NtermProteinFull
2	TRUE	TRUE	GLU,HIS,HIS_D,LYS,PRO,GLN,ARG
3	TRUE	TRUE	GLU,HIS,HIS_D,LYS,GLN,ARG
4	TRUE	TRUE	GLU,HIS,HIS_D,LYS,GLN,ARG
5	TRUE	TRUE	GLU,HIS,HIS_D,LYS,GLN,ARG
6	TRUE	TRUE	GLU,HIS,HIS_D,LYS,GLN,ARG
7	TRUE	TRUE	ALA,ASP,GLU,ILE,LYS,LEU,MET,ASN,GLN,ARG,SER,THR,VAL,TRP,TYR
8	TRUE	TRUE	ALA,ASP,GLU,ILE,LYS,LEU,MET,ASN,GLN,ARG,SER,THR,VAL,TRP,TYR
9	TRUE	TRUE	GLU,HIS,HIS_D,LYS,GLN,ARG
10	TRUE	TRUE	GLU,HIS,HIS_D,LYS,GLN,ARG
11	TRUE	TRUE	ALA,PHE,ILE,LEU,MET,VAL,TRP,TYR
12	TRUE	TRUE	GLU,HIS,HIS_D,LYS,GLN,ARG
13	TRUE	TRUE	GLU,HIS,HIS_D,LYS,GLN,ARG
14	TRUE	TRUE	ALA,PHE,ILE,LEU,MET,VAL,TRP,TYR
15	TRUE	TRUE	ALA,ASP,GLU,ILE,LYS,LEU,MET,ASN,GLN,ARG,SER,THR,VAL,TRP,TYR
16	TRUE	TRUE	GLU,HIS,HIS_D,LYS,GLN,ARG
17	TRUE	TRUE	GLU,HIS,HIS_D,LYS,GLN,ARG
18	TRUE	TRUE	ALA,PHE,ILE,LEU,MET,VAL,TRP,TYR
19	TRUE	TRUE	GLU,HIS,HIS_D,LYS,GLN,ARG
20	TR

The PackerTask looks as intended. Now setup the `MoveMapFactory`:

In [ ]:
# Set up a MoveMapFactory
mmf = pyrosetta.rosetta.core.select.movemap.MoveMapFactory()
mmf.all_bb(setting=False)  # Set to true if needed
mmf.all_bondangles(setting=False)
mmf.all_bondlengths(setting=False)
mmf.all_chi(setting=True)
mmf.all_jumps(setting=False)
mmf.set_cartesian(setting=False)

# Set movemap actions to turn on or off certain torsions, overriding the above defaults
enable = pyrosetta.rosetta.core.select.movemap.move_map_action.mm_enable
disable = pyrosetta.rosetta.core.select.movemap.move_map_action.mm_disable

# Set custom minimizable torsions
mmf.add_bondangles_action(action=enable, selector=chain_A_and_BC_interface)
mmf.add_bondlengths_action(action=enable, selector=chain_A_and_BC_interface)
mmf.add_chi_action(action=enable, selector=chain_A_and_BC_interface)

mmf.add_bondangles_action(action=disable, selector=chain_BC_not_interface)
mmf.add_bondlengths_action(action=disable, selector=chain_BC_not_interface)
mmf.add_chi_action(action=disable, selector=chain_BC_not_interface)

 Now let's double-check some more `pose` information to verify that we are ready for `FastDesign`:

In [ ]:
display_pose = pyrosetta.rosetta.protocols.fold_from_loops.movers.DisplayPoseLabelsMover()
display_pose.tasks(tf)
display_pose.movemap_factory(mmf)
display_pose.apply(pose)

INFO:rosetta:protocols.DsspMover: LHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHLLLLLLLHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHLLLHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHLLLLHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHLLHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHLLLLHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHLL
INFO:rosetta:protocols.fold_from_loops.DisplayPoseLabelsMover: SEQUENCE       AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAYKIKETLKRLEDSLRELRRILEELKEMLERLEKNPDKDVIVEVLKVIVKAIEASVENQRISAENQKALATKYKIKETLKRLEDSLRELRRILEELKEMLERLEKNPDKDVIVEVLKVIVKAIEASVENQRISAENQKALX
INFO:rosetta:protocols.fold_from_loops.DisplayPoseLabelsMover: STRUCTURE      LHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHLLLLLLLHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHLLLHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHLLLLHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHLLHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHLLLLHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHLL
INFO:rosetta:protocols.fold_from_loops.DisplayPoseLabelsMover: MVMP_BB        ...................................................................................................

 We are ready to setup the `FastRelax` mover:

In [ ]:
#fast_design = pyrosetta.rosetta.protocols.denovo_design.movers.FastDesign(scorefxn_in=design_scorefxn, standard_repeats=1) # 2-5 repeats suggested for real applications
fast_design = pyrosetta.rosetta.protocols.relax.FastRelax(scorefxn_in=design_scorefxn, standard_repeats=1, script_file="KillA2019")
fast_design.cartesian(False)
fast_design.set_task_factory(tf)
fast_design.set_movemap_factory(mmf)
fast_design.min_type("lbfgs_armijo_nonmonotone")
fast_design.ramp_down_constraints(False)

INFO:rosetta:protocols.relax.FastRelax: KillA2019 has been renamed to PolarDesign2019
INFO:rosetta:protocols.relax.RelaxScriptManager: score12 : 116.271
INFO:rosetta:protocols.relax.RelaxScriptManager: talaris2013 : 109.417
INFO:rosetta:protocols.relax.RelaxScriptManager: talaris2014 : 109.347
INFO:rosetta:protocols.relax.RelaxScriptManager: ref2015 : 106.875
INFO:rosetta:protocols.relax.RelaxScriptManager: beta_nov16 : 114.938
INFO:rosetta:protocols.relax.RelaxScriptManager: Looking for PolarDesign2019.ref2015.txt
INFO:rosetta:protocols.relax.RelaxScriptManager: ================== Reading script file: /usr/local/lib/python3.7/dist-packages/pyrosetta/database/sampling/relax_scripts/PolarDesign2019.ref2015.txt ==================
INFO:rosetta:protocols.relax.RelaxScriptManager: repeat %%nrepeats%%
INFO:rosetta:protocols.relax.RelaxScriptManager: reference -0.511909 2.0282 -3.29233 -3.74112 2.5917 -1.12843 -1.33724 3.25715 -1.56117 2.65488 2.88076 -2.80685 -1.5498 -2.69754 -1.62133 -1.866

Note that this takes ~37min 13s:

In [ ]:
if not os.getenv("DEBUG"):
    %time fast_design.apply(pose)
    # Optionally, instead of running this you could reload the saved pose from a previously run trajectory:
    pose = pyrosetta.pose_from_file("expected_outputs/designed_pose.pdb")

INFO:rosetta:core.energy_methods.CartesianBondedEnergy: Creating new peptide-bonded energy container (210)
INFO:rosetta:protocols.relax.FastRelax: CMD: repeat  19150.4  0  0  0.55
INFO:rosetta:basic.io.database: Database file opened: scoring/score_functions/aa_repeat_energy/default_repeat_penalty_table.rpt_pen
INFO:rosetta:protocols.relax.FastRelax: CMD: reference  18948.4  0  0  0.55
INFO:rosetta:protocols.relax.FastRelax: CMD: coord_cst_weight  18948.4  0  0  0.55
INFO:rosetta:protocols.relax.FastRelax: CMD: scale:fa_rep  18867.8  0  0  0.03245
INFO:rosetta:core.pack.task: Packer task: initialize from command line()
INFO:rosetta:core.pack.pack_rotamers: built 11290 rotamers at 105 positions.
INFO:rosetta:core.pack.interaction_graph.interaction_graph_factory: Instantiating PDInteractionGraph
INFO:rosetta:protocols.relax.FastRelax: CMD: repack  2534.98  0  0  0.03245


CPU times: user 28min 38s, sys: 7.14 s, total: 28min 45s
Wall time: 28min 40s


RuntimeError: ignored

 Save this pose for downstream reference:

In [ ]:
if not os.getenv("DEBUG"):
    designed_pose = pose.clone()
    #pyrosetta.dump_pdb(designed_pose, "outputs/designed_pose.pdb")

 Now that we have re-designed chain A, it is strongly recommended to use `FastRelax` with a Cartesian scorefunction to repack and minimize with a realistic scorefuction. Note: this takes ~6min 27s

In [ ]:
tf.clear()
tf = pyrosetta.rosetta.core.pack.task.TaskFactory()
tf.push_back(pyrosetta.rosetta.core.pack.task.operation.InitializeFromCommandline())
tf.push_back(pyrosetta.rosetta.core.pack.task.operation.IncludeCurrent())
tf.push_back(pyrosetta.rosetta.core.pack.task.operation.NoRepackDisulfides())
tf.push_back(pyrosetta.rosetta.core.pack.task.operation.OperateOnResidueSubset(
    pyrosetta.rosetta.core.pack.task.operation.RestrictToRepackingRLT(), true_selector))
mm = pyrosetta.rosetta.core.kinematics.MoveMap()
mm.set_bb(False) # Set to true if desired
mm.set_chi(True)
mm.set_jump(False)
fast_relax = pyrosetta.rosetta.protocols.relax.FastRelax(scorefxn_in=relax_scorefxn, standard_repeats=1) # 2-5 repeats suggested for real applications
fast_relax.cartesian(True)
fast_relax.set_task_factory(tf)
fast_relax.set_movemap(mm)
fast_relax.minimize_bond_angles(True)
fast_relax.minimize_bond_lengths(True)
fast_relax.min_type("lbfgs_armijo_nonmonotone") # Cartisian scorefunction
fast_relax.ramp_down_constraints(False)

# To run the FastRelax trajectory.
if not os.getenv("DEBUG"):
    %time fast_relax.apply(pose)

#Or for speed, we will load the pose from a previous trajectory.
pose = pyrosetta.pose_from_file("expected_outputs/designed_relaxed_pose.pdb")

 Save the pose for downstream analysis:

In [ ]:
designed_relaxed_pose = pose.clone()
pyrosetta.dump_pdb(designed_relaxed_pose, "expected_outputs/designed_relaxed_pose.pdb")

 Let's compare sequences to see what happened:

In [ ]:
print(minimized_start_pose.chain_sequence(1))
print(designed_relaxed_pose.chain_sequence(1))

View the resulting design!

In [ ]:
chA = pyrosetta.rosetta.core.select.residue_selector.ChainSelector("A")
chB = pyrosetta.rosetta.core.select.residue_selector.ChainSelector("B")
chC = pyrosetta.rosetta.core.select.residue_selector.ChainSelector("C")
view = sum(
    [
        viewer.init(designed_relaxed_pose),
        viewer.setStyle(cartoon_color="lightgrey", radius=0.25),
        viewer.setSurface(residue_selector=chA, colorscheme="orangeCarbon", opacity=0.5, surface_type="VDW"),
        viewer.setSurface(residue_selector=chB, color="greenCarbon", opacity=0.5, surface_type="VDW"),
        viewer.setSurface(residue_selector=chB, color="violetCarbon", opacity=0.5, surface_type="VDW"),
        viewer.setDisulfides(radius=0.25),
        viewer.setZoom(factor=1.5)
    ]
)
view()

## Analysis:

 By how many Angstroms RMSD did the backbone Cα atoms move?

 What is the delta `total_score` from `minimized_start_pose` to `designed_relaxed_pose`?

 What is the per-residue energy difference for each mutated position between `minimized_start_pose` and `designed_relaxed_pose`?

 Are the sequence constraints imposed by the `aa_repeat` scoreterm satisfied? Re-write the following python code that counts the number of residue types in chain A to check for the longest stretch of each residue type in the primary amino acid sequence in chain A:

In [ ]:
for aa in IUPACData.protein_letters:
    aa_selector = pyrosetta.rosetta.core.select.residue_selector.ResidueNameSelector(str.upper(Bio.SeqUtils.seq3(aa)))
    aa_and_chain_A = pyrosetta.rosetta.core.select.residue_selector.AndResidueSelector(chain_A, aa_selector)
    sel_res_count_metric = pyrosetta.rosetta.core.simple_metrics.metrics.SelectedResidueCountMetric()
    sel_res_count_metric.set_residue_selector(aa_and_chain_A)
    print(aa, int(sel_res_count_metric.calculate(designed_relaxed_pose)))

 Does chain A have 40% percent aromatic or aliphatic (but not leucine) and 5% leucine, satisfying the `aa_composition` scoreterm? For additional practice, re-write the following python implementation using PyRosetta ResidueSelectors and SimpleMetrics:

In [ ]:
num_aro_ali = 0
num_leucine = 0

for aa in pose.chain_sequence(1):
    if aa in "WYFAVIMP":
        num_aro_ali += 1
    if aa == "L":
        num_leucine += 1
        
print("The % aromatic residues in chain A is {0}%".format((num_aro_ali*100)/len(pose.chain_sequence(1))))
print("The % leucine in chain A is {0}%".format((num_leucine*100)/len(pose.chain_sequence(1))))

 Are the sequence constraints imposed by the `AddHelixSequenceConstraints` mover with the `aa_composition` scoreterm satisfied?

 Uses sasa (solvent-accessible surface area) to asses whether there are there more or less voids in `designed_relaxed_pose` as compared to `minimized_start_pose`, satisfying the `voids_penalty` scoreterm.

In [ ]:
tf.clear()
tf = pyrosetta.rosetta.core.pack.task.TaskFactory()
tf.push_back(pyrosetta.rosetta.core.pack.task.operation.OperateOnResidueSubset(
    pyrosetta.rosetta.core.pack.task.operation.RestrictToRepackingRLT(), true_selector))
sasa = pyrosetta.rosetta.protocols.simple_filters.TaskAwareSASAFilter()
sasa.task_factory(tf)
print("The sasa of minimized_start_pose is {}".format(sasa.score(minimized_start_pose)))
print("The sasa of designed_relaxed_pose is {}".format(sasa.score(designed_relaxed_pose)))

 Is the net charge of chain A equal to exactly zero, satisfying the `netcharge` scoreterm?

In [ ]:
# One method to calculate net charge of chain A
num_negative = 0
num_positive = 0
for aa in pose.chain_sequence(1):
    if aa in "DE":
        num_negative += 1
    if aa in "KR":
        num_positive += 1
print("The net charge of chain A = {0}".format(num_positive - num_negative))

# Another method to calculate net charge of chain A
test_pose = pose.clone()
keep_chA = pyrosetta.rosetta.protocols.grafting.simple_movers.KeepRegionMover(res_start=str(test_pose.chain_begin(1)), res_end=str(test_pose.chain_end(1)))
keep_chA.apply(test_pose)
net_charge = pyrosetta.rosetta.protocols.simple_filters.NetChargeFilter()
print("The net charge of chain A = {0}".format(net_charge.score(test_pose)))

 Are there any buried unsatisfied polar atoms, satisfying the `buried_unsatisfied_penalty` scoreterm?

In [ ]:
uhb = pyrosetta.rosetta.protocols.rosetta_scripts.XmlObjects.create_from_string("""
<SCOREFXNS>
  <ScoreFunction name="fa_default" weights="ref2015"/>
</SCOREFXNS>
<FILTERS>
  <BuriedUnsatHbonds name="uhb_sc" use_reporter_behavior="true" use_hbnet_behavior="false" scorefxn="fa_default" report_bb_heavy_atom_unsats="false" report_sc_heavy_atom_unsats="true" cutoff="99999" print_out_info_to_pdb="false" ignore_surface_res="true" residue_surface_cutoff="20.0" ignore_bb_heavy_unsats="false" confidence="0"/>
  <BuriedUnsatHbonds name="uhb_bb" use_reporter_behavior="true" use_hbnet_behavior="false" scorefxn="fa_default" report_bb_heavy_atom_unsats="true" report_sc_heavy_atom_unsats="false" cutoff="99999" print_out_info_to_pdb="false" ignore_surface_res="true" residue_surface_cutoff="20.0" ignore_bb_heavy_unsats="false" confidence="0"/>
</FILTERS>
""")

In [ ]:
print("The number of unsatisfied side-chain heavy atoms in minimized_start_pose is {}".format(uhb.get_filter("uhb_sc").score(minimized_start_pose)))
print("The number of unsatisfied backbone heavy atoms in minimized_start_pose is {}".format(uhb.get_filter("uhb_bb").score(minimized_start_pose)))

print("The number of unsatisfied side-chain heavy atoms in designed_relaxed_pose is {}".format(uhb.get_filter("uhb_sc").score(designed_relaxed_pose)))
print("The number of unsatisfied backbone heavy atoms in designed_relaxed_pose is {}".format(uhb.get_filter("uhb_bb").score(designed_relaxed_pose)))

 Inspect the tracer output from the `BuriedUnsatHbonds` filter, and inspect `designed_relaxed_pose` very closely in PyMol or py3Dmol. Would you agree with the `BuriedUnsatHbonds` filter? How does the number of buried unsatisfied heavy atoms compare to `minimized_start_pose`?

 Packer results are stochastic based on a random number generator, that is after running `pyrosetta.init()` you see:
>rosetta:core.init.random: RandomGenerator:init: Normal mode, seed=937978431 RG_type=mt19937

How do your results compare with your neighbors' results? Ideally you would run the same protocol hundreds of times, and filter them down using Rosetta filters that recaptiulate your design requirements to a number of designs that you can then experimentally validate to answer your biological question.

# See Also

Note these may not be available in PyRosetta through code or even by xml (remodel), but they are extremely useful tools when doing denovo protein design - and you should be aware of them.

- **RosettaRemodel**
 - https://www.rosettacommons.org/docs/latest/application_documentation/design/rosettaremodel
    
    
- **Sewing**

 - https://www.rosettacommons.org/docs/latest/scripting_documentation/RosettaScripts/composite_protocols/sewing/SEWING
 
 
-  **Parametric Design**
 - Previous Workshop!

# **Point Mutation Scan**

Keywords: FastRelax, ResidueSelector, NeighborhoodResidueSelector, TaskFactory, TaskOperation, RestrictToRepackingRLT, RestrictAbsentCanonicalAASRLT, NoRepackDisulfides, OperateOnResidueSubset, RigidBodyTransMover, ggplot2

## Overview
The purpose of this section is to create a protocol using what you have learned in the previous sections.  In this tutorial, we will prepare an antibody-antigen bound structure, make point mutations on the antibody, records changes in binding enregy, and generate a heatmap demonstrating energy differences. This method is widely used in antibody interface design for either improving binding affinity or expanding the binding range. 



## Protocol
The whole protocol is broken down by eight steps.

**Step 1.** Prepare the structure with FastRelax()

**Step 2.** Write the function to perform the mutation PackMover()

**Step 3.** Write the function to unbind the antibody-antigen bound structure unbind()

**Step 4.** Write the function to get wildtype amino acid

**Step 5.** Write the function to properly mutate and pack a specific residue and output energy metrics

**Step 6.** Loop through interface positions mutating them into 20 amino acids with output files

**Step 7.** Summarize all input files for binding energy analysis




## **Section Contributors:**
Yuanhan Wu (The Wistar Institute)

Daniel Kulp (The Wistar Institute)

Jared Adolf-Bryfogle (Scripps; Institute for Protein Innovation)

Ajasja Ljubetič (University of Washington)

### Loading structures 

In [ ]:
from pyrosetta import * 
init()
pose = pose_from_pdb("inputs/1jhl.clean.pdb")
testPose = Pose()
testPose.assign(pose)
print(testPose)

## **Step 1. Prepare the starting structure with FastRelax()**

Properly relaxing a structure is crucial in design with Rosetta. A non relaxed structure may not overcome bad global energy well and therefore skew the following analysis on binding energy.

A FastRelax() is used to relax the structrue. While we want to put the sturcture in its lowest energy state, we want to keep the backbone information from the crystal structure as much as possible (lowest RMSD). Therefore, we apply constrain_relax_to_start_coords(True) to FastRelax().

Since FastRelax() is taking up a huge amount of resource, running it seems to crash the notebook, we commented out the "apply" part (the part that perform the relax) and print out the relaxMover() instead. We uploaded the relaxed structure to the input folder for furthre analysis.

Coordinate constrained relax is a general prep.  For a more advanced version of this preparation method, see the [ pareto-optimal method by Nivon, Morreti, and Baker ](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0059004)



In [ ]:
from pyrosetta.rosetta.protocols.relax import FastRelax

relax = FastRelax()
scorefxn = get_fa_scorefxn()
relax.set_scorefxn(scorefxn)
relax = rosetta.protocols.relax.FastRelax()
relax.constrain_relax_to_start_coords(True)
print(relax)


%time relax.apply(testPose)

testPose.dump_pdb('test.relax.pdb')

### Writing Function in Python

Functions are good ways to organize your code. Starting from this section I am introducing serveral functions to facilitate the protocol.

To define a function in python, a "def" key word is used. A function can either returns a value or simply executing code blocks. A defined function can be called in main function or other sections of code too.

## **Step 2. Write the function for mutation**

This function utilizes the **ResidueSelectorMover()** as demonstrated by previous tutorials. Mutated position is allowed to be designed and repacked while the neighborhood residues are limited to repacked only. The final mutation will be performed with a **PackMover()**.

In [ ]:
from pyrosetta.rosetta.core.pack.task import *
from pyrosetta.rosetta.protocols import *
from pyrosetta.rosetta.core.select import *

def pack(pose, posi, amino, scorefxn):

    # Select Mutate Position
    mut_posi = pyrosetta.rosetta.core.select.residue_selector.ResidueIndexSelector()
    mut_posi.set_index(posi)
    #print(pyrosetta.rosetta.core.select.get_residues_from_subset(mut_posi.apply(pose)))

    # Select Neighbor Position
    nbr_selector = pyrosetta.rosetta.core.select.residue_selector.NeighborhoodResidueSelector()
    nbr_selector.set_focus_selector(mut_posi)
    nbr_selector.set_include_focus_in_subset(True)
    #print(pyrosetta.rosetta.core.select.get_residues_from_subset(nbr_selector.apply(pose)))

    # Select No Design Area
    not_design = pyrosetta.rosetta.core.select.residue_selector.NotResidueSelector(mut_posi)
    #print(pyrosetta.rosetta.core.select.get_residues_from_subset(not_design.apply(pose)))

    # The task factory accepts all the task operations
    tf = pyrosetta.rosetta.core.pack.task.TaskFactory()

    # These are pretty standard
    tf.push_back(pyrosetta.rosetta.core.pack.task.operation.InitializeFromCommandline())
    tf.push_back(pyrosetta.rosetta.core.pack.task.operation.IncludeCurrent())
    tf.push_back(pyrosetta.rosetta.core.pack.task.operation.NoRepackDisulfides())

    # Disable Packing
    prevent_repacking_rlt = pyrosetta.rosetta.core.pack.task.operation.PreventRepackingRLT()
    prevent_subset_repacking = pyrosetta.rosetta.core.pack.task.operation.OperateOnResidueSubset(prevent_repacking_rlt, nbr_selector, True )
    tf.push_back(prevent_subset_repacking)

    # Disable design
    tf.push_back(pyrosetta.rosetta.core.pack.task.operation.OperateOnResidueSubset(
        pyrosetta.rosetta.core.pack.task.operation.RestrictToRepackingRLT(),not_design))

    # Enable design
    aa_to_design = pyrosetta.rosetta.core.pack.task.operation.RestrictAbsentCanonicalAASRLT()
    aa_to_design.aas_to_keep(amino)
    tf.push_back(pyrosetta.rosetta.core.pack.task.operation.OperateOnResidueSubset(aa_to_design, mut_posi))
    
    # Create Packer
    packer = pyrosetta.rosetta.protocols.minimization_packing.PackRotamersMover()
    packer.task_factory(tf)

    #Perform The Move
    if not os.getenv("DEBUG"):
      packer.apply(pose)

#Load the previously-relaxed pose.
relaxPose = pose_from_pdb("inputs/1jhl.relax.pdb")

#Clone it
original = relaxPose.clone()
scorefxn = get_score_function()
print("\nOld Energy:", scorefxn(original),"\n")
pack(relaxPose, 130, 'A', scorefxn)
print("\nNew Energy:", scorefxn(relaxPose),"\n")

#Set relaxPose back to original
relaxPose = original.clone()


### **Step 3. unbind()**
This function is for binding energy analysis. To compute a binding energy, we need to score the total energy of a bound state structure, separate (unbind) the antigen and antibody and then score the unbound state total energy. The binding energy is given by **bound energy** - **unbound energy**.  You can also use the **InterfaceAnalyzerMover (IAM)** , located in `rosetta.protocols.analysis`.

- Benefits to unbind over IAM:
 - Quicker
 - Simpler
 
- Disadvantages to unbind over IAM:
 - Specific foldtree needs to used, with a specific jump. For IAM - it is now foldtree independent, so you only need to know the chainIDs on each side of the interface. 

In [ ]:
from pyrosetta.rosetta.protocols import *

def unbind(pose, partners):
    STEP_SIZE = 100
    JUMP = 2
    docking.setup_foldtree(pose, partners, Vector1([-1,-1,-1]))
    trans_mover = rigid.RigidBodyTransMover(pose,JUMP)
    trans_mover.step_size(STEP_SIZE)
    trans_mover.apply(pose)


##Reset the original pose
relaxPose = original.clone()

scorefxn = get_score_function()
bound_score = scorefxn(relaxPose)
print("\nBound State Score",bound_score,"\n")
unbind(relaxPose, "HL_A")
unbound_score = scorefxn(relaxPose)

print("\nUnbound State Score", unbound_score,"\n")
print('dG', bound_score - unbound_score, 'Rosetta Energy Units (REU)')


## **Step 4. wildtype()**

An important metrics for evaluating binding improvement is the ratio of mutant binding energy to wild type binding energy. This function returns the wild type amino acids in a given position. 

In [ ]:
def wildtype(aatype = 'AA.aa_gly'):
    AA = ['G','A','L','M','F','W','K','Q','E','S','P'
            ,'V','I','C','Y','H','R','N','D','T']

    AA_3 = ['AA.aa_gly','AA.aa_ala','AA.aa_leu','AA.aa_met','AA.aa_phe','AA.aa_trp'
            ,'AA.aa_lys','AA.aa_gln','AA.aa_glu', 'AA.aa_ser','AA.aa_pro','AA.aa_val'
            ,'AA.aa_ile','AA.aa_cys','AA.aa_tyr','AA.aa_his','AA.aa_arg','AA.aa_asn'
            ,'AA.aa_asp','AA.aa_thr']

    for i in range(0, len(AA_3)):
        if(aatype == AA_3[i]):
            return AA[i]

print(wildtype(str(relaxPose.aa(130))))


# Setp 4a.  Exploration. 

We can also use some utility functions instead of this function. 

We can use the sequence STRING returned.  Note that strings are indexed at 0 in Rosetta. 

In [ ]:

print(relaxPose.sequence())
print("\n",relaxPose.sequence()[129])

Now lets get some information fromt the ResidueType object that holds chemical information. 


In [ ]:
print(relaxPose.residue_type(130))

Finally, lets get the residue from the type

In [ ]:
resT = relaxPose.residue_type(130)

print(resT.base_name())
print(resT.name3())
print(resT.name1())

## **Step 5. Integrate functions for mutate and output**

In [ ]:
def mutate(pose, posi, amino, partners):
    #main function for mutation
    CSV_PREFIX = 'notec'
    PDB_PREFIX = 'notep'

    #Initiate test pose
    testPose = Pose()
    testPose.assign(pose)

    #Initiate energy function
    scorefxn = get_fa_scorefxn()
    unbind(testPose, partners)
    native_ub = scorefxn(testPose)
    testPose.assign(pose)
    
    #Variables initiation
    content = ''
    name = CSV_PREFIX + str(posi)+str(amino) + '.csv'
    pdbname = PDB_PREFIX + str(posi)+str(amino) + '.pdb'
    wt = wildtype(str(pose.aa(posi)))

    pack(testPose, posi, amino, scorefxn)
    testPose.dump_pdb(pdbname)
    bound = scorefxn(testPose)
    unbind(testPose, partners)
    unbound = scorefxn(testPose)
    binding = unbound - bound
    testPose.assign(pose)

    if (wt == amino):
        wt_energy = binding
    else:
        pack(testPose, posi, wt, scorefxn)
        wtbound = scorefxn(testPose)
        unbind(testPose, partners)
        wtunbound = scorefxn(testPose)
        wt_energy = wtunbound - wtbound
        testPose.assign(pose)

    content=(content+str(pose.pdb_info().pose2pdb(posi))+','+str(amino)+','
              +str(native_ub)+','+str(bound)+','+str(unbound)+','+str(binding)+','
              +str(wt_energy)+','+str(wt)+','+str(binding/wt_energy)+'\n')

    f = open(name,'w+')
    f.write(content)
    f.close()

mutate(relaxPose, 130, 'A', 'HL_A')


## **Step 6. Loop through interface positions**

The following code loop through all heavy chain and light chain positions mutating them into all 20 amino acids. The actual run may again crash the notebook. 

You may output the data however you wish.  pose.scores() will have the data, maps and pandas Dataframes can also be used.  Here, we will simply output energy information of each mutation in form of csv files and cat them into one file for future analysis. An output file is uploaded to the input folder for the R analysis. 

In real work setting, parallelization can be achieve.  Please see the PyRosetta in Parellel chapter.  For this task, you not necessarily need to finish the whole loop. We have a finished version in the inputs folder.


In [ ]:
#NOTE - This takes a very long time!!  
# You may skip this block to continue the tutorial with pre-configured outputs.
if not os.getenv("DEBUG"):
  for i in [92,85,68,53,5,45,44,42,32,31,22,108,100]:
    print("\nMutating Position: ",str(i),"\n")
    for j in ['G','A','L','M','F','W','K','Q','E','S','P','V','I','C','Y','H','R','N','D','T']:
      mutate(relaxPose, i, j, 'HL_A')

Merging output csv files for binding energy analysis in R

## **Analysis of binding data**
After gathering summarized binding energy information, we use pandas for filtering and visualization. We filtered out lower unbound energy structures, those that have higher unbound state total energy than native and make a heatmap from the filtered data. In case you don't want to finish the for loop at Step 8, we uploaded a finished version of merged output csv to the inputs folder named "note_output.csv".

In [ ]:
#import modules need for analysis
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt 
import seaborn as sns

In [ ]:
csv_file_name = 'inputs/note_output.csv'

In [ ]:
#Generating heatmaps

UNBOUND_CUTOFF = -995
RATIO_CUTOFF = 1.001

#load the data into a pandas dataframe
df = pd.read_csv(csv_file_name, names='Position Amino.Acid Native Bound Unbound Binding WT_Binding WT Ratio'.split(), index_col=False )
#Add wildtype AA to position (for display)
df['Position_WT_aa'] = df.Position + ' (' + df.WT  + ')' 

#filter values
df = df.query(f'Unbound<{UNBOUND_CUTOFF} and Ratio>{RATIO_CUTOFF}')

# convert from tidy format (https://en.wikipedia.org/wiki/Tidy_data) to a wider format
df = pd.pivot_table(df, values='Ratio', 
                     index=['Position_WT_aa'], 
                     columns='Amino.Acid')

#plot the heatmap
f, ax = plt.subplots(figsize=(10, 10))
sns.heatmap(df, cmap='coolwarm', linewidths= 1, linecolor='lightgray')
plt.xlabel('Amino acid mutation');
plt.ylabel('Position chain and wild type AA');
sns.set_context("talk") #make labels bigger

# See Also

Note these may not be available in PyRosetta through code or even by xml (remodel), but they are extremely useful tools when doing denovo protein design - and you should be aware of them.

- **RosettaRemodel**
 - https://www.rosettacommons.org/docs/latest/application_documentation/design/rosettaremodel
    
    
- **Sewing**

 - https://www.rosettacommons.org/docs/latest/scripting_documentation/RosettaScripts/composite_protocols/sewing/SEWING
 
 
-  **Parametric Design**
 - Previous Workshop!